In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import time
import os
from datetime import datetime

from model_final import Model
from configs_final import config
from tensorflow.core.protobuf import rewriter_config_pb2
from sklearn.model_selection import train_test_split

In [2]:
# First train the network without any regularization like batch norm, dropout and L2 loss
NUM_CLASS = 4
MAX_STEP = 1000
LOG_FREQUENCY = 10
BATCH_SIZE = 50
MODEL_SAVING_FREQUENCY = 100
TARGET_DIR = './trained_model_final/DCNN'

def data_loader():
    X_test = np.load("X_test.npy")
    X_test = X_test[:,0:22,:]
    y_test = np.load("y_test.npy")
    person_test = np.load("person_test.npy")

    X_train_valid = np.load("X_train_valid.npy")
    X_train_valid = X_train_valid[:,0:22,:]
    y_train_valid = np.load("y_train_valid.npy")
    person_train_valid = np.load("person_train_valid.npy")
    N_train = X_train_valid.shape[0]
    idx_train_valid = np.arange(N_train,dtype='int')

    X_train, X_val, idx_train, idx_val = train_test_split(X_train_valid, idx_train_valid, test_size=0.1, random_state=21)

    y_train = y_train_valid[idx_train]
    person_train = person_train_valid[idx_train]
    y_val = y_train_valid[idx_val]
    person_val = person_train_valid[idx_val]

    tl = X_train.shape[2]
    X_test = np.reshape(X_test,(-1,22,tl,1))
    X_train = np.reshape(X_train,(-1,22,tl,1))
    X_val = np.reshape(X_val,(-1,22,tl,1))

    label0 = 769
    new_label0 = 0
    for i in range(4):
        m1 = (y_test==label0)
        m2 = (y_train==label0)
        m3 = (y_val==label0)
        np.place(y_test,m1,new_label0)
        np.place(y_train,m2,new_label0)
        np.place(y_val,m3,new_label0)
        label0 += 1
        new_label0 += 1

    labelNames = [0,1,2,3]
    train_set = {'data': X_train, 'labels': y_train, 'person': person_train}
    test_set = {'data': X_test, 'labels': y_test, 'person': person_test}
    val_set = {'data': X_val, 'labels': y_val, 'person': person_val}
    return train_set, test_set, val_set, labelNames


def sample_batch(dataset, batch_size):
    N = dataset['data'].shape[0]
    indices = np.random.randint(N, size=batch_size)
    return {key: dataset[key][indices] for key in dataset}


def main():
    # load data
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader()
    print("Dataset loading completes.")

    # reset default graph
    tf.reset_default_graph()

    global_step = tf.train.get_or_create_global_step()
    
    start = 0
    stop = 1000
    step = 100
    time_length = 1000
    #Nb = BATCH_SIZE
    fsz = 10
    
    model = Model(config, labelNames,start,stop,step,time_length,fsz,False,False,False)

    # training setups
    saver = tf.train.Saver(max_to_keep=100)

    # Build an initialization operation to run below.
    init = tf.global_variables_initializer()
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off
    
    sess = tf.Session(config=config_proto)

    sess.run(init)

    # main loop
    for step in range(MAX_STEP):
        start_time = time.time()

        # prepare data
        train_batch = sample_batch(trainSet, BATCH_SIZE)
        # feed dict
        feed_dict = {
            model.input: train_batch['data'],
            model.fine_labels: train_batch['labels'],
            model.is_training: True,
        }
        
        fetch_list = [model.optimizer_op, model.loss, model.global_step]
        _, loss_value, global_step_value = sess.run(fetch_list, feed_dict=feed_dict)
        duration = time.time() - start_time
        assert not np.isnan(loss_value), 'Model diverged with loss = NaN'

        # log
        if step % LOG_FREQUENCY == 0:
            num_examples_per_step = BATCH_SIZE
            examples_per_sec = num_examples_per_step / duration
            sec_per_batch = duration
            format_str = (
                '%s: step %d, examples %d, loss = %.9f (%.3f examples/sec; %.3f sec/batch)'
            )
            print(
                format_str % (
                    datetime.now(), step, BATCH_SIZE * step,
                    loss_value,
                    examples_per_sec, sec_per_batch
                )
            )

        # Save the model checkpoint periodically.
        if step % MODEL_SAVING_FREQUENCY == 0 or (step + 1) == MAX_STEP:
            fetch_list = [model.accuracy,model.top_2_accuracy]
            feed_dict = {
                model.input: valSet['data'],
                model.fine_labels: valSet['labels'],
                model.is_training: False,
            }
            val_acc_1, val_acc_2 =  sess.run(fetch_list, feed_dict=feed_dict)
            checkpoint_path = os.path.join(TARGET_DIR, 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step=int(global_step_value))
            print('Top 1 validation accuracy: {} and top 2 validation accuracy: {}'.format(val_acc_1,val_acc_2))
            print('Model Saved!')


if __name__ == '__main__':
    main()

Loading datasets...
Dataset loading completes.
2019-03-15 19:03:03.584374: step 0, examples 0, loss = 1.402904868 (90.105 examples/sec; 0.555 sec/batch)
Top 1 validation accuracy: 0.24528302252292633 and top 2 validation accuracy: 0.4716981053352356
Model Saved!
2019-03-15 19:03:08.861117: step 10, examples 500, loss = 1.399699807 (125.609 examples/sec; 0.398 sec/batch)
2019-03-15 19:03:12.880040: step 20, examples 1000, loss = 1.266509414 (126.967 examples/sec; 0.394 sec/batch)
2019-03-15 19:03:16.878071: step 30, examples 1500, loss = 1.260570168 (127.698 examples/sec; 0.392 sec/batch)
2019-03-15 19:03:20.877470: step 40, examples 2000, loss = 1.254686236 (123.123 examples/sec; 0.406 sec/batch)
2019-03-15 19:03:24.869723: step 50, examples 2500, loss = 1.077540159 (125.505 examples/sec; 0.398 sec/batch)
2019-03-15 19:03:29.031716: step 60, examples 3000, loss = 1.024056911 (127.986 examples/sec; 0.391 sec/batch)
2019-03-15 19:03:33.097981: step 70, examples 3500, loss = 1.062388897 (

2019-03-15 19:07:42.909110: step 660, examples 33000, loss = 0.000351356 (122.771 examples/sec; 0.407 sec/batch)
2019-03-15 19:07:47.120058: step 670, examples 33500, loss = 0.000244107 (120.223 examples/sec; 0.416 sec/batch)
2019-03-15 19:07:51.189097: step 680, examples 34000, loss = 0.000255331 (122.915 examples/sec; 0.407 sec/batch)
2019-03-15 19:07:55.251528: step 690, examples 34500, loss = 0.000268399 (122.738 examples/sec; 0.407 sec/batch)
2019-03-15 19:07:59.319860: step 700, examples 35000, loss = 0.000209941 (125.823 examples/sec; 0.397 sec/batch)
Top 1 validation accuracy: 0.5094339847564697 and top 2 validation accuracy: 0.75
Model Saved!
2019-03-15 19:08:04.859868: step 710, examples 35500, loss = 0.000192922 (119.184 examples/sec; 0.420 sec/batch)
2019-03-15 19:08:09.185992: step 720, examples 36000, loss = 0.000202350 (115.477 examples/sec; 0.433 sec/batch)
2019-03-15 19:08:13.426671: step 730, examples 36500, loss = 0.000141824 (113.162 examples/sec; 0.442 sec/batch)
2

In [4]:
#best model is 601

def test_model(model_dir,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss):
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader()
    print("Dataset loading completes.")
    
    # reset default graph
    tf.reset_default_graph()
    
    # create model
    
    model = Model(config, labelNames,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss)
    # training setups
    saver = tf.train.Saver(max_to_keep=100)
    test_accuracy = [0, 0]
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off

    with tf.Session(config=config_proto) as session:
        session.run(tf.global_variables_initializer())
        saver.restore(session, model_dir)
        # test model
        exp_results = run_single_step(session, model, testSet, mode='test')
        test_accuracy[0] = exp_results['top_1_accuracy']
        test_accuracy[1] = exp_results['top_2_accuracy']
        print('top_1_accuracy_test = ', test_accuracy[0], 'top_2_accuracy_test = ', test_accuracy[1])
        
    top1_acc = test_accuracy[0]
    top2_acc = test_accuracy[1]
    return top1_acc,top2_acc


def run_single_step(
        session,
        model,
        batch,
        mode='test',
        log=True,
):
    # construct feed dict
    feed_dict = {
        model.input: batch['data'],
        # model.coarse_labels: batch['coarse_labels'],
        model.fine_labels: batch['labels'],
        # model.label_mapping: label_mapping,
        model.is_training: mode == 'train'
    }
    
    # select proper summary op
    if mode == 'train':
        summary_op = model.train_summary_op
    elif mode == 'val':
        summary_op = model.val_summary_op
    else:
        summary_op = model.test_summary_op
    
    # construct fetch list
    fetch_list = [model.global_step, summary_op, model.loss, model.accuracy, model.top_2_accuracy]

    # run single step
    _start_time = time.time()
    _step, _summary, _loss, _top_1, _top_2 = session.run(fetch_list, feed_dict=feed_dict)[:5]
    _end_time = time.time()
    
    # collect step statistics
    step_time = _end_time - _start_time
    batch_size = batch['data'].shape[0]
    
    # log in console
    if log:
        print(('[{:5s} step {:4d}] loss: {:.5f}; top_1_accuracy: {:.5f}; top_5_accuracy: {:5f} ' +
              '({:.3f} sec/batch; {:.3f} instances/sec)'
              ).format(mode, _step, _loss, _top_1, _top_2, 
                       step_time, batch_size / step_time))
    
    # log results to file and return statistics
    if mode == 'test':
        test_fetch_list = [model.per_class_accuracy,
                model.top_2_per_class_accuracy,
                model.confusion_matrix, 
                model.pred, model.probs]
        _top_1_c,  _top_2_c, _cm, _pred, _probs = \
                session.run(test_fetch_list, feed_dict=feed_dict)
        
        
        # Log detailed test results in pickle format
        stats = {
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2,
            "top_1_perclass_accuracy": _top_1_c,
            "top_2_perclass_accuracy": _top_2_c,
            "confusion_matrix": _cm,
            "pred": _pred,
            "probs": _probs
        }
    else:
        stats = {
            "step": _step,
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2
        }
        
    return stats

In [5]:
# Accuracy for testSet 601
model_dir = "./trained_model_final/DCNN/model.ckpt-601"
start = 0
stop = 1000
step = 100
time_length = 1000
#Nb = BATCH_SIZE
fsz = 10
use_batchnorm = False
use_dropout = False
use_l2loss = False
top_1_acc_org, top_2_acc_org = test_model(model_dir,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss)

Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN/model.ckpt-601
[test  step  601] loss: 2.56059; top_1_accuracy: 0.50339; top_5_accuracy: 0.742664 (1.309 sec/batch; 338.319 instances/sec)
top_1_accuracy_test =  0.503386 top_2_accuracy_test =  0.7426637


In [6]:
# Now train the network with regularization like batch norm, dropout and L2 loss

def train_model(target_dir,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss,num_class,max_step,log_frequency,batch_size,model_saving_freq):
    # load data
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader()
    print("Dataset loading completes.")

    # reset default graph
    tf.reset_default_graph()

    global_step = tf.train.get_or_create_global_step()
    
    model = Model(config, labelNames,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss)

    # training setups
    saver = tf.train.Saver(max_to_keep=100)

    # Build an initialization operation to run below.
    init = tf.global_variables_initializer()
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off
    
    sess = tf.Session(config=config_proto)

    sess.run(init)

    # main loop
    for step in range(max_step):
        start_time = time.time()

        # prepare data
        train_batch = sample_batch(trainSet, batch_size)
        # feed dict
        feed_dict = {
            model.input: train_batch['data'],
            model.fine_labels: train_batch['labels'],
            model.is_training: True,
        }
        
        fetch_list = [model.optimizer_op, model.loss, model.global_step]
        _, loss_value, global_step_value = sess.run(fetch_list, feed_dict=feed_dict)
        duration = time.time() - start_time
        assert not np.isnan(loss_value), 'Model diverged with loss = NaN'

        # log
        if step % log_frequency == 0:
            num_examples_per_step = batch_size
            examples_per_sec = num_examples_per_step / duration
            sec_per_batch = duration
            format_str = (
                '%s: step %d, examples %d, loss = %.9f (%.3f examples/sec; %.3f sec/batch)'
            )
            print(
                format_str % (
                    datetime.now(), step, batch_size * step,
                    loss_value,
                    examples_per_sec, sec_per_batch
                )
            )

        # Save the model checkpoint periodically.
        if step % model_saving_freq == 0 or (step + 1) == max_step:
            fetch_list = [model.accuracy,model.top_2_accuracy]
            feed_dict = {
                model.input: valSet['data'],
                model.fine_labels: valSet['labels'],
                model.is_training: False,
            }
            val_acc_1, val_acc_2 =  sess.run(fetch_list, feed_dict=feed_dict)
            checkpoint_path = os.path.join(target_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step=int(global_step_value))
            print('Top 1 validation accuracy: {} and top 2 validation accuracy: {}'.format(val_acc_1,val_acc_2))
            print('Model Saved!')


In [7]:
num_cls = 4
mstp = 1000
lfrq = 10
bsz = 50
msf = 100
tr = './trained_model_final/DCNN_reg'
start = 0
stop = 1000
step = 100
time_length = 1000
#Nb = BATCH_SIZE
fsz = 10
use_batchnorm = True
use_dropout = True
use_l2loss = True

train_model(tr,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss,num_cls,mstp,lfrq,bsz,msf)

Loading datasets...
Dataset loading completes.
2019-03-15 19:36:07.429616: step 0, examples 0, loss = 1.481527090 (54.813 examples/sec; 0.912 sec/batch)
Top 1 validation accuracy: 0.25471699237823486 and top 2 validation accuracy: 0.5235849022865295
Model Saved!
2019-03-15 19:36:15.043898: step 10, examples 500, loss = 1.477544785 (85.081 examples/sec; 0.588 sec/batch)
2019-03-15 19:36:20.684522: step 20, examples 1000, loss = 1.462527990 (89.579 examples/sec; 0.558 sec/batch)
2019-03-15 19:36:26.170413: step 30, examples 1500, loss = 1.493336678 (89.194 examples/sec; 0.561 sec/batch)
2019-03-15 19:36:31.706731: step 40, examples 2000, loss = 1.439875841 (91.459 examples/sec; 0.547 sec/batch)
2019-03-15 19:36:37.218895: step 50, examples 2500, loss = 1.430225611 (87.385 examples/sec; 0.572 sec/batch)
2019-03-15 19:36:42.769315: step 60, examples 3000, loss = 1.317382336 (88.507 examples/sec; 0.565 sec/batch)
2019-03-15 19:36:48.289031: step 70, examples 3500, loss = 1.422132969 (86.902

2019-03-15 19:42:49.202727: step 670, examples 33500, loss = 0.805280328 (86.687 examples/sec; 0.577 sec/batch)
2019-03-15 19:42:55.064397: step 680, examples 34000, loss = 0.844516695 (84.299 examples/sec; 0.593 sec/batch)
2019-03-15 19:43:01.042173: step 690, examples 34500, loss = 0.708914340 (84.777 examples/sec; 0.590 sec/batch)
2019-03-15 19:43:06.877369: step 700, examples 35000, loss = 0.760557890 (86.675 examples/sec; 0.577 sec/batch)
Top 1 validation accuracy: 0.5801886916160583 and top 2 validation accuracy: 0.7877358198165894
Model Saved!
2019-03-15 19:43:14.703428: step 710, examples 35500, loss = 0.615862072 (83.993 examples/sec; 0.595 sec/batch)
2019-03-15 19:43:20.578438: step 720, examples 36000, loss = 0.644081771 (84.260 examples/sec; 0.593 sec/batch)
2019-03-15 19:43:26.473737: step 730, examples 36500, loss = 0.639435589 (83.843 examples/sec; 0.596 sec/batch)
2019-03-15 19:43:32.324773: step 740, examples 37000, loss = 0.626901567 (88.301 examples/sec; 0.566 sec/ba

In [8]:
# Accuracy for testSet 701 (best)
model_dir = "./trained_model_final/DCNN_reg/model.ckpt-701"
start = 0
stop = 1000
step = 100
time_length = 1000
#Nb = BATCH_SIZE
fsz = 10
use_batchnorm = True
use_dropout = True
use_l2loss = True
top_1_acc_org, top_2_acc_org = test_model(model_dir,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss)

Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg/model.ckpt-701
[test  step  701] loss: 1.49420; top_1_accuracy: 0.56433; top_5_accuracy: 0.821670 (1.523 sec/batch; 290.838 instances/sec)
top_1_accuracy_test =  0.5643341 top_2_accuracy_test =  0.8216704


In [9]:
# Train only with dropout and l2loss

num_cls = 4
mstp = 1000
lfrq = 10
bsz = 50
msf = 100
tr = './trained_model_final/DCNN_reg2'
start = 0
stop = 1000
step = 100
time_length = 1000
#Nb = BATCH_SIZE
fsz = 10
use_batchnorm = False
use_dropout = True
use_l2loss = True

train_model(tr,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss,num_cls,mstp,lfrq,bsz,msf)

Loading datasets...
Dataset loading completes.
2019-03-15 19:51:37.806159: step 0, examples 0, loss = 1.487277269 (78.597 examples/sec; 0.636 sec/batch)
Top 1 validation accuracy: 0.21226415038108826 and top 2 validation accuracy: 0.49528300762176514
Model Saved!
2019-03-15 19:51:43.366185: step 10, examples 500, loss = 1.483107328 (113.203 examples/sec; 0.442 sec/batch)
2019-03-15 19:51:47.489193: step 20, examples 1000, loss = 1.461261034 (120.840 examples/sec; 0.414 sec/batch)
2019-03-15 19:51:51.564133: step 30, examples 1500, loss = 1.452936292 (122.075 examples/sec; 0.410 sec/batch)
2019-03-15 19:51:55.659489: step 40, examples 2000, loss = 1.500891209 (117.816 examples/sec; 0.424 sec/batch)
2019-03-15 19:51:59.791915: step 50, examples 2500, loss = 1.391779542 (118.295 examples/sec; 0.423 sec/batch)
2019-03-15 19:52:03.904646: step 60, examples 3000, loss = 1.565071821 (124.353 examples/sec; 0.402 sec/batch)
2019-03-15 19:52:08.000272: step 70, examples 3500, loss = 1.323044777 

2019-03-15 19:56:30.217126: step 660, examples 33000, loss = 0.493227005 (109.913 examples/sec; 0.455 sec/batch)
2019-03-15 19:56:34.641083: step 670, examples 33500, loss = 0.557910204 (114.547 examples/sec; 0.437 sec/batch)
2019-03-15 19:56:38.940073: step 680, examples 34000, loss = 0.682563126 (118.506 examples/sec; 0.422 sec/batch)
2019-03-15 19:56:43.299240: step 690, examples 34500, loss = 0.549979508 (113.831 examples/sec; 0.439 sec/batch)
2019-03-15 19:56:47.675686: step 700, examples 35000, loss = 0.484500051 (111.325 examples/sec; 0.449 sec/batch)
Top 1 validation accuracy: 0.5660377144813538 and top 2 validation accuracy: 0.7735849022865295
Model Saved!
2019-03-15 19:56:53.657416: step 710, examples 35500, loss = 0.626695752 (109.765 examples/sec; 0.456 sec/batch)
2019-03-15 19:56:58.001059: step 720, examples 36000, loss = 0.615614533 (114.274 examples/sec; 0.438 sec/batch)
2019-03-15 19:57:02.356982: step 730, examples 36500, loss = 0.835541368 (115.120 examples/sec; 0.43

In [10]:
# Accuracy for testSet 901 (best)
model_dir = "./trained_model_final/DCNN_reg2/model.ckpt-901"
start = 0
stop = 1000
step = 100
time_length = 1000
#Nb = BATCH_SIZE
fsz = 10
use_batchnorm = False
use_dropout = True
use_l2loss = True
top_1_acc_org, top_2_acc_org = test_model(model_dir,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss)

Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg2/model.ckpt-901
[test  step  901] loss: 1.70193; top_1_accuracy: 0.54402; top_5_accuracy: 0.830700 (1.343 sec/batch; 329.870 instances/sec)
top_1_accuracy_test =  0.54401803 top_2_accuracy_test =  0.8306998


## Batch Norm or No Batch Norm
Adding batch normalization gave 2% better result on the test dataset, even though the validation accuracy without batch norm was higher. Which weakly implies that the model with batch norm is more generalizable

In [13]:
def data_loader_tb(time_bin):
    X_test = np.load("X_test.npy")
    X_test = X_test[:,0:22,0:time_bin]
    y_test = np.load("y_test.npy")
    person_test = np.load("person_test.npy")

    X_train_valid = np.load("X_train_valid.npy")
    X_train_valid = X_train_valid[:,0:22,0:time_bin]
    y_train_valid = np.load("y_train_valid.npy")
    person_train_valid = np.load("person_train_valid.npy")
    
    N_train = X_train_valid.shape[0]
    idx_train_valid = np.arange(N_train,dtype='int')

    X_train, X_val, idx_train, idx_val = train_test_split(X_train_valid, idx_train_valid, test_size=0.1, random_state=21)

    y_train = y_train_valid[idx_train]
    person_train = person_train_valid[idx_train]
    y_val = y_train_valid[idx_val]
    person_val = person_train_valid[idx_val]

    tl = X_train.shape[2]
    X_test = np.reshape(X_test,(-1,22,tl,1))
    X_train = np.reshape(X_train,(-1,22,tl,1))
    X_val = np.reshape(X_val,(-1,22,tl,1))

    label0 = 769
    new_label0 = 0
    for i in range(4):
        m1 = (y_test==label0)
        m2 = (y_train==label0)
        m3 = (y_val==label0)
        np.place(y_test,m1,new_label0)
        np.place(y_train,m2,new_label0)
        np.place(y_val,m3,new_label0)
        label0 += 1
        new_label0 += 1

    labelNames = [0,1,2,3]
    train_set = {'data': X_train, 'labels': y_train, 'person': person_train}
    test_set = {'data': X_test, 'labels': y_test, 'person': person_test}
    val_set = {'data': X_val, 'labels': y_val, 'person': person_val}
    return train_set, test_set, val_set, labelNames


def sample_batch(dataset, batch_size):
    N = dataset['data'].shape[0]
    indices = np.random.randint(N, size=batch_size)
    return {key: dataset[key][indices] for key in dataset}

def train_model_tb(target_dir,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss,num_class,max_step,log_frequency,batch_size,model_saving_freq):
    # load data
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader_tb(time_length)
    print("Dataset loading completes.")
    
    best_val_acc = 0
    best_val_path = target_dir

    # reset default graph
    tf.reset_default_graph()

    global_step = tf.train.get_or_create_global_step()
    
    model = Model(config, labelNames,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss)

    # training setups
    saver = tf.train.Saver(max_to_keep=100)

    # Build an initialization operation to run below.
    init = tf.global_variables_initializer()
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off
    
    sess = tf.Session(config=config_proto)

    sess.run(init)

    # main loop
    for step in range(max_step):
        start_time = time.time()

        # prepare data
        train_batch = sample_batch(trainSet, batch_size)
        # feed dict
        feed_dict = {
            model.input: train_batch['data'],
            model.fine_labels: train_batch['labels'],
            model.is_training: True,
        }
        
        fetch_list = [model.optimizer_op, model.loss, model.global_step]
        _, loss_value, global_step_value = sess.run(fetch_list, feed_dict=feed_dict)
        duration = time.time() - start_time
        assert not np.isnan(loss_value), 'Model diverged with loss = NaN'

        # log
        if step % log_frequency == 0:
            num_examples_per_step = batch_size
            examples_per_sec = num_examples_per_step / duration
            sec_per_batch = duration
            format_str = (
                '%s: step %d, examples %d, loss = %.9f (%.3f examples/sec; %.3f sec/batch)'
            )
            print(
                format_str % (
                    datetime.now(), step, batch_size * step,
                    loss_value,
                    examples_per_sec, sec_per_batch
                )
            )

        # Save the model checkpoint periodically.
        if step % model_saving_freq == 0 or (step + 1) == max_step:
            fetch_list = [model.accuracy,model.top_2_accuracy]
            feed_dict = {
                model.input: valSet['data'],
                model.fine_labels: valSet['labels'],
                model.is_training: False,
            }
            val_acc_1, val_acc_2 =  sess.run(fetch_list, feed_dict=feed_dict)
            checkpoint_path = os.path.join(target_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step=int(global_step_value))
            print('Top 1 validation accuracy: {} and top 2 validation accuracy: {}'.format(val_acc_1,val_acc_2))
            print('Model Saved!')
            if val_acc_1>best_val_acc:
                best_val_acc = val_acc_1
                best_val_path = checkpoint_path+'-'+str(int(global_step_value))
    return best_val_acc,best_val_path


In [16]:
import os
filter_size = [8,8,9,9,10,10]
j=0
best_val = {}
for tl in range(500,1001,100):
    num_cls = 4
    mstp = 1000
    lfrq = 10
    bsz = 50
    msf = 100
    tr = './trained_model_final/DCNN_reg_t'+str(tl)
    os.mkdir(tr)
    start = 0
    stop = tl
    step = 100
    time_length = tl
    fsz = filter_size[j]
    j = j+1
    use_batchnorm = True
    use_dropout = True
    use_l2loss = True
    acc = 'Accuracy'+str(tl)
    path = 'Path'+str(tl)

    best_val[acc],best_val[path] = train_model_tb(tr,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss,num_cls,mstp,lfrq,bsz,msf)

Loading datasets...
Dataset loading completes.
2019-03-15 21:52:58.930062: step 0, examples 0, loss = 1.475036025 (82.492 examples/sec; 0.606 sec/batch)
Top 1 validation accuracy: 0.2358490526676178 and top 2 validation accuracy: 0.4858490526676178
Model Saved!
2019-03-15 21:53:03.216090: step 10, examples 500, loss = 1.491670966 (184.279 examples/sec; 0.271 sec/batch)
2019-03-15 21:53:05.907360: step 20, examples 1000, loss = 1.548984766 (188.215 examples/sec; 0.266 sec/batch)
2019-03-15 21:53:08.538250: step 30, examples 1500, loss = 1.437868714 (184.191 examples/sec; 0.271 sec/batch)
2019-03-15 21:53:11.200766: step 40, examples 2000, loss = 1.451322913 (183.810 examples/sec; 0.272 sec/batch)
2019-03-15 21:53:13.829003: step 50, examples 2500, loss = 1.292630792 (188.215 examples/sec; 0.266 sec/batch)
2019-03-15 21:53:16.471125: step 60, examples 3000, loss = 1.257043123 (194.109 examples/sec; 0.258 sec/batch)
2019-03-15 21:53:19.173365: step 70, examples 3500, loss = 1.464167833 (1

2019-03-15 21:56:18.463414: step 660, examples 33000, loss = 0.659628868 (169.726 examples/sec; 0.295 sec/batch)
2019-03-15 21:56:21.362489: step 670, examples 33500, loss = 0.604616165 (178.569 examples/sec; 0.280 sec/batch)
2019-03-15 21:56:24.204267: step 680, examples 34000, loss = 0.510358334 (178.731 examples/sec; 0.280 sec/batch)
2019-03-15 21:56:27.283217: step 690, examples 34500, loss = 0.519466817 (144.962 examples/sec; 0.345 sec/batch)
2019-03-15 21:56:30.658595: step 700, examples 35000, loss = 0.618541837 (147.974 examples/sec; 0.338 sec/batch)
Top 1 validation accuracy: 0.5754716992378235 and top 2 validation accuracy: 0.7971698045730591
Model Saved!
2019-03-15 21:56:35.199590: step 710, examples 35500, loss = 0.488366753 (178.240 examples/sec; 0.281 sec/batch)
2019-03-15 21:56:38.021206: step 720, examples 36000, loss = 0.672968268 (177.759 examples/sec; 0.281 sec/batch)
2019-03-15 21:56:40.834429: step 730, examples 36500, loss = 0.644783795 (178.739 examples/sec; 0.28

2019-03-15 21:59:58.643291: step 310, examples 15500, loss = 0.960688233 (139.802 examples/sec; 0.358 sec/batch)
2019-03-15 22:00:02.017183: step 320, examples 16000, loss = 0.938169897 (159.983 examples/sec; 0.313 sec/batch)
2019-03-15 22:00:05.409517: step 330, examples 16500, loss = 0.900954366 (157.094 examples/sec; 0.318 sec/batch)
2019-03-15 22:00:08.766698: step 340, examples 17000, loss = 1.204232574 (149.306 examples/sec; 0.335 sec/batch)
2019-03-15 22:00:12.155567: step 350, examples 17500, loss = 1.036954045 (153.317 examples/sec; 0.326 sec/batch)
2019-03-15 22:00:15.502543: step 360, examples 18000, loss = 1.007626653 (151.803 examples/sec; 0.329 sec/batch)
2019-03-15 22:00:18.951640: step 370, examples 18500, loss = 0.922007024 (145.439 examples/sec; 0.344 sec/batch)
2019-03-15 22:00:22.393287: step 380, examples 19000, loss = 1.065659761 (155.200 examples/sec; 0.322 sec/batch)
2019-03-15 22:00:25.769527: step 390, examples 19500, loss = 0.835670173 (153.917 examples/sec; 

2019-03-15 22:03:58.220443: step 980, examples 49000, loss = 0.456899583 (152.029 examples/sec; 0.329 sec/batch)
2019-03-15 22:04:01.645290: step 990, examples 49500, loss = 0.551029921 (146.505 examples/sec; 0.341 sec/batch)
Top 1 validation accuracy: 0.599056601524353 and top 2 validation accuracy: 0.8113207817077637
Model Saved!
Loading datasets...
Dataset loading completes.
2019-03-15 22:04:09.173694: step 0, examples 0, loss = 1.457754254 (66.741 examples/sec; 0.749 sec/batch)
Top 1 validation accuracy: 0.25 and top 2 validation accuracy: 0.5094339847564697
Model Saved!
2019-03-15 22:04:15.015896: step 10, examples 500, loss = 1.435146809 (123.064 examples/sec; 0.406 sec/batch)
2019-03-15 22:04:19.124772: step 20, examples 1000, loss = 1.479582429 (123.989 examples/sec; 0.403 sec/batch)
2019-03-15 22:04:23.188870: step 30, examples 1500, loss = 1.428952456 (118.824 examples/sec; 0.421 sec/batch)
2019-03-15 22:04:27.420458: step 40, examples 2000, loss = 1.454817295 (118.982 exampl

2019-03-15 22:08:40.061385: step 630, examples 31500, loss = 0.812939823 (127.987 examples/sec; 0.391 sec/batch)
2019-03-15 22:08:44.173870: step 640, examples 32000, loss = 0.541201711 (114.768 examples/sec; 0.436 sec/batch)
2019-03-15 22:08:48.343328: step 650, examples 32500, loss = 0.511578560 (111.903 examples/sec; 0.447 sec/batch)
2019-03-15 22:08:52.486368: step 660, examples 33000, loss = 0.583886147 (118.230 examples/sec; 0.423 sec/batch)
2019-03-15 22:08:56.545020: step 670, examples 33500, loss = 0.701379061 (124.126 examples/sec; 0.403 sec/batch)
2019-03-15 22:09:00.562676: step 680, examples 34000, loss = 0.653021753 (128.455 examples/sec; 0.389 sec/batch)
2019-03-15 22:09:04.749982: step 690, examples 34500, loss = 0.590762079 (109.067 examples/sec; 0.458 sec/batch)
2019-03-15 22:09:09.019854: step 700, examples 35000, loss = 0.469384104 (125.446 examples/sec; 0.399 sec/batch)
Top 1 validation accuracy: 0.5707547068595886 and top 2 validation accuracy: 0.7877358198165894


2019-03-15 22:13:37.781221: step 290, examples 14500, loss = 1.034456730 (113.460 examples/sec; 0.441 sec/batch)
2019-03-15 22:13:42.378236: step 300, examples 15000, loss = 1.153218985 (109.835 examples/sec; 0.455 sec/batch)
Top 1 validation accuracy: 0.5235849022865295 and top 2 validation accuracy: 0.7311320900917053
Model Saved!
2019-03-15 22:13:48.640868: step 310, examples 15500, loss = 0.950036705 (110.710 examples/sec; 0.452 sec/batch)
2019-03-15 22:13:53.232592: step 320, examples 16000, loss = 1.031529307 (112.770 examples/sec; 0.443 sec/batch)
2019-03-15 22:13:57.812841: step 330, examples 16500, loss = 1.183319926 (110.927 examples/sec; 0.451 sec/batch)
2019-03-15 22:14:02.357236: step 340, examples 17000, loss = 1.121684432 (112.222 examples/sec; 0.446 sec/batch)
2019-03-15 22:14:06.987679: step 350, examples 17500, loss = 1.265745997 (103.105 examples/sec; 0.485 sec/batch)
2019-03-15 22:14:11.594551: step 360, examples 18000, loss = 0.892851532 (110.435 examples/sec; 0.45

2019-03-15 22:18:53.378321: step 950, examples 47500, loss = 0.593931675 (109.886 examples/sec; 0.455 sec/batch)
2019-03-15 22:18:58.032071: step 960, examples 48000, loss = 0.503998160 (106.192 examples/sec; 0.471 sec/batch)
2019-03-15 22:19:02.656743: step 970, examples 48500, loss = 0.649743676 (103.570 examples/sec; 0.483 sec/batch)
2019-03-15 22:19:07.213523: step 980, examples 49000, loss = 0.677944779 (108.547 examples/sec; 0.461 sec/batch)
2019-03-15 22:19:11.844939: step 990, examples 49500, loss = 0.552101672 (103.522 examples/sec; 0.483 sec/batch)
Top 1 validation accuracy: 0.5235849022865295 and top 2 validation accuracy: 0.7735849022865295
Model Saved!
Loading datasets...
Dataset loading completes.
2019-03-15 22:19:20.896782: step 0, examples 0, loss = 1.542200089 (55.802 examples/sec; 0.896 sec/batch)
Top 1 validation accuracy: 0.24528302252292633 and top 2 validation accuracy: 0.47641509771347046
Model Saved!
2019-03-15 22:19:28.157799: step 10, examples 500, loss = 1.50

2019-03-15 22:24:56.102590: step 610, examples 30500, loss = 0.678909361 (92.681 examples/sec; 0.539 sec/batch)
2019-03-15 22:25:01.330833: step 620, examples 31000, loss = 0.684697509 (96.758 examples/sec; 0.517 sec/batch)
2019-03-15 22:25:06.641424: step 630, examples 31500, loss = 1.054907441 (95.699 examples/sec; 0.522 sec/batch)
2019-03-15 22:25:11.893247: step 640, examples 32000, loss = 0.826445699 (91.662 examples/sec; 0.545 sec/batch)
2019-03-15 22:25:17.141859: step 650, examples 32500, loss = 0.660133183 (96.956 examples/sec; 0.516 sec/batch)
2019-03-15 22:25:22.378328: step 660, examples 33000, loss = 0.578632772 (95.224 examples/sec; 0.525 sec/batch)
2019-03-15 22:25:27.640841: step 670, examples 33500, loss = 0.499891728 (94.309 examples/sec; 0.530 sec/batch)
2019-03-15 22:25:32.812971: step 680, examples 34000, loss = 0.491972238 (94.347 examples/sec; 0.530 sec/batch)
2019-03-15 22:25:38.057030: step 690, examples 34500, loss = 0.696641862 (98.628 examples/sec; 0.507 sec

2019-03-15 22:31:21.594444: step 280, examples 14000, loss = 1.238872290 (84.446 examples/sec; 0.592 sec/batch)
2019-03-15 22:31:27.424649: step 290, examples 14500, loss = 1.075294256 (81.725 examples/sec; 0.612 sec/batch)
2019-03-15 22:31:33.351918: step 300, examples 15000, loss = 0.924402058 (87.676 examples/sec; 0.570 sec/batch)
Top 1 validation accuracy: 0.5188679099082947 and top 2 validation accuracy: 0.7452830076217651
Model Saved!
2019-03-15 22:31:41.113515: step 310, examples 15500, loss = 1.294166446 (88.998 examples/sec; 0.562 sec/batch)
2019-03-15 22:31:47.075166: step 320, examples 16000, loss = 1.055491447 (87.201 examples/sec; 0.573 sec/batch)
2019-03-15 22:31:52.891173: step 330, examples 16500, loss = 0.970112503 (90.493 examples/sec; 0.553 sec/batch)
2019-03-15 22:31:58.703465: step 340, examples 17000, loss = 0.888216138 (83.822 examples/sec; 0.597 sec/batch)
2019-03-15 22:32:04.626060: step 350, examples 17500, loss = 1.259779334 (84.809 examples/sec; 0.590 sec/ba

2019-03-15 22:38:06.891025: step 950, examples 47500, loss = 0.623467922 (89.465 examples/sec; 0.559 sec/batch)
2019-03-15 22:38:12.799268: step 960, examples 48000, loss = 0.585890710 (84.008 examples/sec; 0.595 sec/batch)
2019-03-15 22:38:18.638560: step 970, examples 48500, loss = 0.461982131 (81.788 examples/sec; 0.611 sec/batch)
2019-03-15 22:38:24.561037: step 980, examples 49000, loss = 0.425914586 (76.439 examples/sec; 0.654 sec/batch)
2019-03-15 22:38:30.408691: step 990, examples 49500, loss = 0.570369422 (85.755 examples/sec; 0.583 sec/batch)
Top 1 validation accuracy: 0.5896226167678833 and top 2 validation accuracy: 0.8066037893295288
Model Saved!


In [17]:
#best model is 601

def test_model_tb(model_dir,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss):
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader_tb(time_length)
    print("Dataset loading completes.")
    
    # reset default graph
    tf.reset_default_graph()
    
    # create model
    
    model = Model(config, labelNames,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss)
    # training setups
    saver = tf.train.Saver(max_to_keep=100)
    test_accuracy = [0, 0]
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off

    with tf.Session(config=config_proto) as session:
        session.run(tf.global_variables_initializer())
        saver.restore(session, model_dir)
        # test model
        exp_results = run_single_step(session, model, testSet, mode='test')
        test_accuracy[0] = exp_results['top_1_accuracy']
        test_accuracy[1] = exp_results['top_2_accuracy']
        print('top_1_accuracy_test = ', test_accuracy[0], 'top_2_accuracy_test = ', test_accuracy[1])
        
    top1_acc = test_accuracy[0]
    top2_acc = test_accuracy[1]
    return top1_acc,top2_acc


def run_single_step(
        session,
        model,
        batch,
        mode='test',
        log=True,
):
    # construct feed dict
    feed_dict = {
        model.input: batch['data'],
        # model.coarse_labels: batch['coarse_labels'],
        model.fine_labels: batch['labels'],
        # model.label_mapping: label_mapping,
        model.is_training: mode == 'train'
    }
    
    # select proper summary op
    if mode == 'train':
        summary_op = model.train_summary_op
    elif mode == 'val':
        summary_op = model.val_summary_op
    else:
        summary_op = model.test_summary_op
    
    # construct fetch list
    fetch_list = [model.global_step, summary_op, model.loss, model.accuracy, model.top_2_accuracy]

    # run single step
    _start_time = time.time()
    _step, _summary, _loss, _top_1, _top_2 = session.run(fetch_list, feed_dict=feed_dict)[:5]
    _end_time = time.time()
    
    # collect step statistics
    step_time = _end_time - _start_time
    batch_size = batch['data'].shape[0]
    
    # log in console
    if log:
        print(('[{:5s} step {:4d}] loss: {:.5f}; top_1_accuracy: {:.5f}; top_5_accuracy: {:5f} ' +
              '({:.3f} sec/batch; {:.3f} instances/sec)'
              ).format(mode, _step, _loss, _top_1, _top_2, 
                       step_time, batch_size / step_time))
    
    # log results to file and return statistics
    if mode == 'test':
        test_fetch_list = [model.per_class_accuracy,
                model.top_2_per_class_accuracy,
                model.confusion_matrix, 
                model.pred, model.probs]
        _top_1_c,  _top_2_c, _cm, _pred, _probs = \
                session.run(test_fetch_list, feed_dict=feed_dict)
        
        
        # Log detailed test results in pickle format
        stats = {
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2,
            "top_1_perclass_accuracy": _top_1_c,
            "top_2_perclass_accuracy": _top_2_c,
            "confusion_matrix": _cm,
            "pred": _pred,
            "probs": _probs
        }
    else:
        stats = {
            "step": _step,
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2
        }
        
    return stats

In [22]:
#best model is 601

def val_model_tb(model_dir,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss):
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader_tb(time_length)
    print("Dataset loading completes.")
    
    # reset default graph
    tf.reset_default_graph()
    
    # create model
    
    model = Model(config, labelNames,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss)
    # training setups
    saver = tf.train.Saver(max_to_keep=100)
    val_accuracy = [0, 0]
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off

    with tf.Session(config=config_proto) as session:
        session.run(tf.global_variables_initializer())
        saver.restore(session, model_dir)
        # test model
        exp_results = run_single_step(session, model, valSet, mode='val')
        val_accuracy[0] = exp_results['top_1_accuracy']
        val_accuracy[1] = exp_results['top_2_accuracy']
        print('top_1_accuracy_val = ', val_accuracy[0], 'top_2_accuracy_val = ', val_accuracy[1])
        
    top1_acc = val_accuracy[0]
    top2_acc = val_accuracy[1]
    return top1_acc,top2_acc


def run_single_step(
        session,
        model,
        batch,
        mode='test',
        log=True,
):
    # construct feed dict
    feed_dict = {
        model.input: batch['data'],
        # model.coarse_labels: batch['coarse_labels'],
        model.fine_labels: batch['labels'],
        # model.label_mapping: label_mapping,
        model.is_training: mode == 'train'
    }
    
    # select proper summary op
    if mode == 'train':
        summary_op = model.train_summary_op
    elif mode == 'val':
        summary_op = model.val_summary_op
    else:
        summary_op = model.test_summary_op
    
    # construct fetch list
    fetch_list = [model.global_step, summary_op, model.loss, model.accuracy, model.top_2_accuracy]

    # run single step
    _start_time = time.time()
    _step, _summary, _loss, _top_1, _top_2 = session.run(fetch_list, feed_dict=feed_dict)[:5]
    _end_time = time.time()
    
    # collect step statistics
    step_time = _end_time - _start_time
    batch_size = batch['data'].shape[0]
    
    # log in console
    if log:
        print(('[{:5s} step {:4d}] loss: {:.5f}; top_1_accuracy: {:.5f}; top_5_accuracy: {:5f} ' +
              '({:.3f} sec/batch; {:.3f} instances/sec)'
              ).format(mode, _step, _loss, _top_1, _top_2, 
                       step_time, batch_size / step_time))
    
    # log results to file and return statistics
    if mode == 'test':
        test_fetch_list = [model.per_class_accuracy,
                model.top_2_per_class_accuracy,
                model.confusion_matrix, 
                model.pred, model.probs]
        _top_1_c,  _top_2_c, _cm, _pred, _probs = \
                session.run(test_fetch_list, feed_dict=feed_dict)
        
        
        # Log detailed test results in pickle format
        stats = {
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2,
            "top_1_perclass_accuracy": _top_1_c,
            "top_2_perclass_accuracy": _top_2_c,
            "confusion_matrix": _cm,
            "pred": _pred,
            "probs": _probs
        }
    else:
        stats = {
            "step": _step,
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2
        }
        
    return stats

In [23]:
best_val = {}
mod_num= np.arange(1,902,100)
ns = mod_num.shape
mod_num = np.insert(mod_num,ns,1000,axis=0)
filter_size = [8,8,9,9,10,10]
j=0
for tl in range(500,1001,100):
    acc = 'Accuracy'+str(tl)
    path = 'Path'+str(tl)
    start = 0
    stop = tl
    step = 100
    time_length = tl
    #Nb = BATCH_SIZE
    fsz = filter_size[j]
    j = j+1
    use_batchnorm = True
    use_dropout = True
    use_l2loss = True
    best_val_acc = 0
    best_val_path = '.'
    for mn in mod_num:
        model_dir = './trained_model_final/DCNN_reg_t'+str(tl)+'/model.ckpt-'+str(mn)
        top_1_acc_val, top_2_acc_val = val_model_tb(model_dir,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss)
        if top_1_acc_val>best_val_acc:
            best_val_acc = top_1_acc_val
            best_val_path = model_dir
    best_val[acc] = best_val_acc
    best_val[path] = best_val_path

Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_t500/model.ckpt-1
[val   step    1] loss: 1.47676; top_1_accuracy: 0.20755; top_5_accuracy: 0.490566 (0.445 sec/batch; 476.317 instances/sec)
top_1_accuracy_val =  0.20754717 top_2_accuracy_val =  0.49056605
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_t500/model.ckpt-101
[val   step  101] loss: 1.39558; top_1_accuracy: 0.41509; top_5_accuracy: 0.674528 (0.430 sec/batch; 492.657 instances/sec)
top_1_accuracy_val =  0.41509435 top_2_accuracy_val =  0.6745283
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_t500/model.ckpt-201
[val   step  201] loss: 1.28704; top_1_accuracy: 0.49057; top_5_accuracy: 0.740566 (0.426 sec/batch; 497.584 instances/sec)
top_1_accuracy_val =  0.49056605 top_2_accuracy_val =  0.740566
Loading dataset

Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_t700/model.ckpt-301
[val   step  301] loss: 1.26042; top_1_accuracy: 0.51887; top_5_accuracy: 0.764151 (0.543 sec/batch; 390.532 instances/sec)
top_1_accuracy_val =  0.5188679 top_2_accuracy_val =  0.7641509
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_t700/model.ckpt-401
[val   step  401] loss: 1.22889; top_1_accuracy: 0.55660; top_5_accuracy: 0.792453 (0.591 sec/batch; 358.979 instances/sec)
top_1_accuracy_val =  0.5566038 top_2_accuracy_val =  0.7924528
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_t700/model.ckpt-501
[val   step  501] loss: 1.36093; top_1_accuracy: 0.53774; top_5_accuracy: 0.759434 (0.551 sec/batch; 384.965 instances/sec)
top_1_accuracy_val =  0.5377358 top_2_accuracy_val =  0.759434
Loading datasets...
Dataset loading c

INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_t900/model.ckpt-601
[val   step  601] loss: 1.44636; top_1_accuracy: 0.54717; top_5_accuracy: 0.759434 (0.706 sec/batch; 300.459 instances/sec)
top_1_accuracy_val =  0.5471698 top_2_accuracy_val =  0.759434
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_t900/model.ckpt-701
[val   step  701] loss: 1.66897; top_1_accuracy: 0.50472; top_5_accuracy: 0.750000 (0.704 sec/batch; 301.283 instances/sec)
top_1_accuracy_val =  0.504717 top_2_accuracy_val =  0.75
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_t900/model.ckpt-801
[val   step  801] loss: 1.68642; top_1_accuracy: 0.54245; top_5_accuracy: 0.759434 (0.686 sec/batch; 309.089 instances/sec)
top_1_accuracy_val =  0.5424528 top_2_accuracy_val =  0.759434
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restorin

In [28]:
# Accuracy for testSet(best)
filter_size = [8,8,9,9,10,10]
j=0
best_test_acc = 0
best_test_time_bin = 500
for tl in range(500,1001,100):
    path = 'Path'+str(tl)
    model_dir = best_val[path]
    start = 0
    stop = tl
    step = 100
    time_length = tl
    #Nb = BATCH_SIZE
    fsz = filter_size[j]
    j = j+1
    use_batchnorm = True
    use_dropout = True
    use_l2loss = True
    print('Accuracy for Time bin: {}'.format(tl))
    top_1_acc_test, top_2_acc_test = test_model_tb(model_dir,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss)
    if top_1_acc_test > best_test_acc:
        best_test_acc = top_1_acc_test
        best_test_time_bin = tl

print('The time bin {} has the best test accuracy: {}'.format(best_test_time_bin,best_test_acc))

Accuracy for Time bin: 500
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_t500/model.ckpt-601
[test  step  601] loss: 1.29574; top_1_accuracy: 0.54853; top_5_accuracy: 0.778781 (0.806 sec/batch; 549.542 instances/sec)
top_1_accuracy_test =  0.5485327 top_2_accuracy_test =  0.77878106
Accuracy for Time bin: 600
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_t600/model.ckpt-701
[test  step  701] loss: 1.30332; top_1_accuracy: 0.57336; top_5_accuracy: 0.830700 (0.953 sec/batch; 464.765 instances/sec)
top_1_accuracy_test =  0.5733634 top_2_accuracy_test =  0.8306998
Accuracy for Time bin: 700
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_t700/model.ckpt-601
[test  step  601] loss: 1.38312; top_1_accuracy: 0.58239; top_5_accuracy: 0.819413 (1.121 sec/batch; 395.199 instances

In [38]:
def data_loader_ds(time_bin,ds):
    X_test = np.load("X_test.npy")
    X_test = X_test[:,0:22,0:time_bin:ds]
    y_test = np.load("y_test.npy")
    person_test = np.load("person_test.npy")

    X_train_valid = np.load("X_train_valid.npy")
    X_train_valid = X_train_valid[:,0:22,0:time_bin:ds]
    y_train_valid = np.load("y_train_valid.npy")
    person_train_valid = np.load("person_train_valid.npy")
    
    N_train = X_train_valid.shape[0]
    idx_train_valid = np.arange(N_train,dtype='int')

    X_train, X_val, idx_train, idx_val = train_test_split(X_train_valid, idx_train_valid, test_size=0.1, random_state=21)

    y_train = y_train_valid[idx_train]
    person_train = person_train_valid[idx_train]
    y_val = y_train_valid[idx_val]
    person_val = person_train_valid[idx_val]

    tl = X_train.shape[2]
    X_test = np.reshape(X_test,(-1,22,tl,1))
    X_train = np.reshape(X_train,(-1,22,tl,1))
    X_val = np.reshape(X_val,(-1,22,tl,1))

    label0 = 769
    new_label0 = 0
    for i in range(4):
        m1 = (y_test==label0)
        m2 = (y_train==label0)
        m3 = (y_val==label0)
        np.place(y_test,m1,new_label0)
        np.place(y_train,m2,new_label0)
        np.place(y_val,m3,new_label0)
        label0 += 1
        new_label0 += 1

    labelNames = [0,1,2,3]
    train_set = {'data': X_train, 'labels': y_train, 'person': person_train}
    test_set = {'data': X_test, 'labels': y_test, 'person': person_test}
    val_set = {'data': X_val, 'labels': y_val, 'person': person_val}
    return train_set, test_set, val_set, labelNames


def sample_batch(dataset, batch_size):
    N = dataset['data'].shape[0]
    indices = np.random.randint(N, size=batch_size)
    return {key: dataset[key][indices] for key in dataset}

def train_model_ds(target_dir,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss,num_class,max_step,log_frequency,batch_size,model_saving_freq,ds):
    # load data
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader_ds(time_length,ds)
    print("Dataset loading completes.")
    
    best_val_acc = 0
    best_val_path = target_dir
    
    time_bin = trainSet['data'].shape[2]
    stop = trainSet['data'].shape[2]

    # reset default graph
    tf.reset_default_graph()

    global_step = tf.train.get_or_create_global_step()
    
    model = Model(config, labelNames,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)

    # training setups
    saver = tf.train.Saver(max_to_keep=100)

    # Build an initialization operation to run below.
    init = tf.global_variables_initializer()
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off
    
    sess = tf.Session(config=config_proto)

    sess.run(init)

    # main loop
    for step in range(max_step):
        start_time = time.time()

        # prepare data
        train_batch = sample_batch(trainSet, batch_size)
        # feed dict
        feed_dict = {
            model.input: train_batch['data'],
            model.fine_labels: train_batch['labels'],
            model.is_training: True,
        }
        
        fetch_list = [model.optimizer_op, model.loss, model.global_step]
        _, loss_value, global_step_value = sess.run(fetch_list, feed_dict=feed_dict)
        duration = time.time() - start_time
        assert not np.isnan(loss_value), 'Model diverged with loss = NaN'

        # log
        if step % log_frequency == 0:
            num_examples_per_step = batch_size
            examples_per_sec = num_examples_per_step / duration
            sec_per_batch = duration
            format_str = (
                '%s: step %d, examples %d, loss = %.9f (%.3f examples/sec; %.3f sec/batch)'
            )
            print(
                format_str % (
                    datetime.now(), step, batch_size * step,
                    loss_value,
                    examples_per_sec, sec_per_batch
                )
            )

        # Save the model checkpoint periodically.
        if step % model_saving_freq == 0 or (step + 1) == max_step:
            fetch_list = [model.accuracy,model.top_2_accuracy]
            feed_dict = {
                model.input: valSet['data'],
                model.fine_labels: valSet['labels'],
                model.is_training: False,
            }
            val_acc_1, val_acc_2 =  sess.run(fetch_list, feed_dict=feed_dict)
            checkpoint_path = os.path.join(target_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step=int(global_step_value))
            print('Top 1 validation accuracy: {} and top 2 validation accuracy: {}'.format(val_acc_1,val_acc_2))
            print('Model Saved!')
            if val_acc_1>best_val_acc:
                best_val_acc = val_acc_1
                best_val_path = checkpoint_path+'-'+str(int(global_step_value))
    return best_val_acc,best_val_path


In [40]:
filter_size = [9,6,4]
j=0
best_val_ds = {}
time_bin = 700
for ds in range(1,4,1):
    num_cls = 4
    mstp = 1000
    lfrq = 10
    bsz = 50
    msf = 100
    tr = './trained_model_final/DCNN_reg_ds'+str(ds)
    if not os.path.exists(tr):
        os.mkdir(tr)
    start = 0
    stop = time_bin
    step = 100
    time_length = time_bin
    fsz = filter_size[j]
    j = j+1
    use_batchnorm = True
    use_dropout = True
    use_l2loss = True
    acc = 'Accuracy'+str(ds)
    path = 'Path'+str(ds)

    best_val_ds[acc],best_val_ds[path] = train_model_ds(tr,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss,num_cls,mstp,lfrq,bsz,msf,ds)

Loading datasets...
Dataset loading completes.
2019-03-16 02:40:51.252357: step 0, examples 0, loss = 1.458157659 (61.909 examples/sec; 0.808 sec/batch)
Top 1 validation accuracy: 0.2358490526676178 and top 2 validation accuracy: 0.4858490526676178
Model Saved!
2019-03-16 02:40:56.888053: step 10, examples 500, loss = 1.511277199 (130.592 examples/sec; 0.383 sec/batch)
2019-03-16 02:41:00.755084: step 20, examples 1000, loss = 1.507617474 (133.732 examples/sec; 0.374 sec/batch)
2019-03-16 02:41:04.675550: step 30, examples 1500, loss = 1.431611300 (128.111 examples/sec; 0.390 sec/batch)
2019-03-16 02:41:08.536811: step 40, examples 2000, loss = 1.411809087 (127.407 examples/sec; 0.392 sec/batch)
2019-03-16 02:41:12.365177: step 50, examples 2500, loss = 1.353659272 (132.486 examples/sec; 0.377 sec/batch)
2019-03-16 02:41:16.225225: step 60, examples 3000, loss = 1.414247036 (127.619 examples/sec; 0.392 sec/batch)
2019-03-16 02:41:20.113713: step 70, examples 3500, loss = 1.285899758 (1

2019-03-16 02:45:30.770720: step 660, examples 33000, loss = 0.496823847 (117.611 examples/sec; 0.425 sec/batch)
2019-03-16 02:45:35.200501: step 670, examples 33500, loss = 0.660404384 (111.435 examples/sec; 0.449 sec/batch)
2019-03-16 02:45:39.608723: step 680, examples 34000, loss = 0.595655501 (115.835 examples/sec; 0.432 sec/batch)
2019-03-16 02:45:43.925734: step 690, examples 34500, loss = 0.505445361 (113.326 examples/sec; 0.441 sec/batch)
2019-03-16 02:45:48.307514: step 700, examples 35000, loss = 0.630921066 (114.769 examples/sec; 0.436 sec/batch)
Top 1 validation accuracy: 0.6084905862808228 and top 2 validation accuracy: 0.801886796951294
Model Saved!
2019-03-16 02:45:54.562798: step 710, examples 35500, loss = 0.591579676 (117.334 examples/sec; 0.426 sec/batch)
2019-03-16 02:45:58.869822: step 720, examples 36000, loss = 0.582153082 (114.374 examples/sec; 0.437 sec/batch)
2019-03-16 02:46:03.225921: step 730, examples 36500, loss = 0.666428566 (120.598 examples/sec; 0.415

2019-03-16 02:49:12.325212: step 310, examples 15500, loss = 0.982916772 (256.386 examples/sec; 0.195 sec/batch)
2019-03-16 02:49:14.280892: step 320, examples 16000, loss = 0.949410200 (251.220 examples/sec; 0.199 sec/batch)
2019-03-16 02:49:16.263545: step 330, examples 16500, loss = 0.815233529 (249.961 examples/sec; 0.200 sec/batch)
2019-03-16 02:49:18.208215: step 340, examples 17000, loss = 1.089880228 (261.083 examples/sec; 0.192 sec/batch)
2019-03-16 02:49:20.187979: step 350, examples 17500, loss = 1.046365619 (261.083 examples/sec; 0.192 sec/batch)
2019-03-16 02:49:22.126151: step 360, examples 18000, loss = 0.974002361 (259.725 examples/sec; 0.193 sec/batch)
2019-03-16 02:49:24.044764: step 370, examples 18500, loss = 1.156610847 (265.250 examples/sec; 0.189 sec/batch)
2019-03-16 02:49:25.957851: step 380, examples 19000, loss = 0.722630262 (263.847 examples/sec; 0.190 sec/batch)
2019-03-16 02:49:27.920570: step 390, examples 19500, loss = 0.826190054 (254.424 examples/sec; 

2019-03-16 02:51:37.621277: step 980, examples 49000, loss = 0.730834365 (261.083 examples/sec; 0.192 sec/batch)
2019-03-16 02:51:39.572967: step 990, examples 49500, loss = 0.789062202 (254.424 examples/sec; 0.197 sec/batch)
Top 1 validation accuracy: 0.5754716992378235 and top 2 validation accuracy: 0.8160377144813538
Model Saved!
Loading datasets...
Dataset loading completes.
2019-03-16 02:51:45.545011: step 0, examples 0, loss = 1.427079678 (97.778 examples/sec; 0.511 sec/batch)
Top 1 validation accuracy: 0.22641509771347046 and top 2 validation accuracy: 0.5
Model Saved!
2019-03-16 02:51:48.336579: step 10, examples 500, loss = 1.429236650 (405.423 examples/sec; 0.123 sec/batch)
2019-03-16 02:51:49.598936: step 20, examples 1000, loss = 1.432451844 (403.780 examples/sec; 0.124 sec/batch)
2019-03-16 02:51:50.870317: step 30, examples 1500, loss = 1.429117441 (370.758 examples/sec; 0.135 sec/batch)
2019-03-16 02:51:52.381353: step 40, examples 2000, loss = 1.394075155 (323.815 examp

2019-03-16 02:53:17.507266: step 630, examples 31500, loss = 0.843805730 (398.936 examples/sec; 0.125 sec/batch)
2019-03-16 02:53:18.786168: step 640, examples 32000, loss = 0.883151293 (389.586 examples/sec; 0.128 sec/batch)
2019-03-16 02:53:20.107180: step 650, examples 32500, loss = 0.941706359 (392.653 examples/sec; 0.127 sec/batch)
2019-03-16 02:53:21.379062: step 660, examples 33000, loss = 0.928657711 (395.774 examples/sec; 0.126 sec/batch)
2019-03-16 02:53:22.655456: step 670, examples 33500, loss = 0.864767730 (397.349 examples/sec; 0.126 sec/batch)
2019-03-16 02:53:23.940373: step 680, examples 34000, loss = 0.987384140 (402.155 examples/sec; 0.124 sec/batch)
2019-03-16 02:53:25.211265: step 690, examples 34500, loss = 0.796272159 (392.651 examples/sec; 0.127 sec/batch)
2019-03-16 02:53:26.487187: step 700, examples 35000, loss = 0.901980758 (379.215 examples/sec; 0.132 sec/batch)
Top 1 validation accuracy: 0.5 and top 2 validation accuracy: 0.75
Model Saved!
2019-03-16 02:53

In [41]:

def test_model_ds(model_dir,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss,ds):
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader_ds(time_length,ds)
    print("Dataset loading completes.")
    
    time_bin = testSet['data'].shape[2]
    stop = testSet['data'].shape[2]
    
    # reset default graph
    tf.reset_default_graph()
    
    # create model
    
    model = Model(config, labelNames,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)
    # training setups
    saver = tf.train.Saver(max_to_keep=100)
    test_accuracy = [0, 0]
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off

    with tf.Session(config=config_proto) as session:
        session.run(tf.global_variables_initializer())
        saver.restore(session, model_dir)
        # test model
        exp_results = run_single_step(session, model, testSet, mode='test')
        test_accuracy[0] = exp_results['top_1_accuracy']
        test_accuracy[1] = exp_results['top_2_accuracy']
        print('top_1_accuracy_test = ', test_accuracy[0], 'top_2_accuracy_test = ', test_accuracy[1])
        
    top1_acc = test_accuracy[0]
    top2_acc = test_accuracy[1]
    return top1_acc,top2_acc


def run_single_step(
        session,
        model,
        batch,
        mode='test',
        log=True,
):
    # construct feed dict
    feed_dict = {
        model.input: batch['data'],
        # model.coarse_labels: batch['coarse_labels'],
        model.fine_labels: batch['labels'],
        # model.label_mapping: label_mapping,
        model.is_training: mode == 'train'
    }
    
    # select proper summary op
    if mode == 'train':
        summary_op = model.train_summary_op
    elif mode == 'val':
        summary_op = model.val_summary_op
    else:
        summary_op = model.test_summary_op
    
    # construct fetch list
    fetch_list = [model.global_step, summary_op, model.loss, model.accuracy, model.top_2_accuracy]

    # run single step
    _start_time = time.time()
    _step, _summary, _loss, _top_1, _top_2 = session.run(fetch_list, feed_dict=feed_dict)[:5]
    _end_time = time.time()
    
    # collect step statistics
    step_time = _end_time - _start_time
    batch_size = batch['data'].shape[0]
    
    # log in console
    if log:
        print(('[{:5s} step {:4d}] loss: {:.5f}; top_1_accuracy: {:.5f}; top_5_accuracy: {:5f} ' +
              '({:.3f} sec/batch; {:.3f} instances/sec)'
              ).format(mode, _step, _loss, _top_1, _top_2, 
                       step_time, batch_size / step_time))
    
    # log results to file and return statistics
    if mode == 'test':
        test_fetch_list = [model.per_class_accuracy,
                model.top_2_per_class_accuracy,
                model.confusion_matrix, 
                model.pred, model.probs]
        _top_1_c,  _top_2_c, _cm, _pred, _probs = \
                session.run(test_fetch_list, feed_dict=feed_dict)
        
        
        # Log detailed test results in pickle format
        stats = {
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2,
            "top_1_perclass_accuracy": _top_1_c,
            "top_2_perclass_accuracy": _top_2_c,
            "confusion_matrix": _cm,
            "pred": _pred,
            "probs": _probs
        }
    else:
        stats = {
            "step": _step,
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2
        }
        
    return stats

In [42]:
# Accuracy for testSet(best)
filter_size = [9,6,4]
j=0
time_bin = 700
best_test_acc_ds = 0
best_test_ds = 0
for ds in range(1,4,1):
    path = 'Path'+str(ds)
    model_dir = best_val_ds[path]
    start = 0
    stop = time_bin
    step = 100
    time_length = time_bin
    #Nb = BATCH_SIZE
    fsz = filter_size[j]
    j = j+1
    use_batchnorm = True
    use_dropout = True
    use_l2loss = True
    print('Accuracy for Downsampling rate: {}'.format(ds))
    top_1_acc_test, top_2_acc_test = test_model_ds(model_dir,start,stop,step,time_length,fsz,use_batchnorm,use_dropout,use_l2loss,ds)
    if top_1_acc_test > best_test_acc_ds:
        best_test_acc_ds = top_1_acc_test
        best_test_ds = ds

print('The downsampling rate {} has the best test accuracy: {}'.format(best_test_ds,best_test_acc_ds))

Accuracy for Downsampling rate: 1
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ds1\model.ckpt-701
[test  step  701] loss: 1.41690; top_1_accuracy: 0.58691; top_5_accuracy: 0.790068 (1.089 sec/batch; 406.972 instances/sec)
top_1_accuracy_test =  0.58690745 top_2_accuracy_test =  0.79006773
Accuracy for Downsampling rate: 2
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ds2\model.ckpt-501
[test  step  501] loss: 1.20136; top_1_accuracy: 0.55079; top_5_accuracy: 0.810384 (0.583 sec/batch; 760.341 instances/sec)
top_1_accuracy_test =  0.5507901 top_2_accuracy_test =  0.81038374
Accuracy for Downsampling rate: 3
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ds3\model.ckpt-501
[test  step  501] loss: 1.25981; top_1_accuracy: 0.50790; top_5_accuracy: 0.753950 (0.387 sec/batc

In [45]:
#downsample and concatenate to increase the data size

def data_loader_ss(time_bin):
    X_test = np.load("X_test.npy")
    X_test = X_test[:,0:22,0:time_bin]
    y_test = np.load("y_test.npy")
    person_test = np.load("person_test.npy")

    X_train_valid = np.load("X_train_valid.npy")
    X_train_valid = X_train_valid[:,0:22,0:time_bin]
    y_train_valid = np.load("y_train_valid.npy")
    person_train_valid = np.load("person_train_valid.npy")
    
    N_train = X_train_valid.shape[0]
    idx_train_valid = np.arange(N_train,dtype='int')

    X_train, X_val, idx_train, idx_val = train_test_split(X_train_valid, idx_train_valid, test_size=0.1, random_state=21)

    y_train = y_train_valid[idx_train]
    person_train = person_train_valid[idx_train]
    y_val = y_train_valid[idx_val]
    person_val = person_train_valid[idx_val]
    
    idx11 = np.arange(0,X_test.shape[2],2,dtype=int)
    idx21 = np.arange(1,X_test.shape[2],2,dtype=int)
    X_test_ss1 = np.take(X_test,idx11,axis=2)
    X_test_ss2 = np.take(X_test,idx21,axis=2)
    X_test = np.concatenate((X_test_ss1,X_test_ss2),axis=0)
    y_test = np.concatenate((y_test,y_test),axis=0)
    person_test = np.concatenate((person_test,person_test),axis=0)
    
    idx12 = np.arange(0,X_train.shape[2],2,dtype=int)
    idx22 = np.arange(1,X_train.shape[2],2,dtype=int)
    X_train_ss1 = np.take(X_train,idx12,axis=2)
    X_train_ss2 = np.take(X_train,idx22,axis=2)
    X_train = np.concatenate((X_train_ss1,X_train_ss2),axis=0)
    y_train = np.concatenate((y_train,y_train),axis=0)
    person_train = np.concatenate((person_train,person_train),axis=0)
    
    idx13 = np.arange(0,X_val.shape[2],2,dtype=int)
    idx23 = np.arange(1,X_val.shape[2],2,dtype=int)
    X_val_ss1 = np.take(X_val,idx13,axis=2)
    X_val_ss2 = np.take(X_val,idx23,axis=2)
    X_val = np.concatenate((X_val_ss1,X_val_ss2),axis=0)
    y_val = np.concatenate((y_val,y_val),axis=0)
    person_val = np.concatenate((person_val,person_val),axis=0)

    tl = X_train.shape[2]
    X_test = np.reshape(X_test,(-1,22,tl,1))
    X_train = np.reshape(X_train,(-1,22,tl,1))
    X_val = np.reshape(X_val,(-1,22,tl,1))

    label0 = 769
    new_label0 = 0
    for i in range(4):
        m1 = (y_test==label0)
        m2 = (y_train==label0)
        m3 = (y_val==label0)
        np.place(y_test,m1,new_label0)
        np.place(y_train,m2,new_label0)
        np.place(y_val,m3,new_label0)
        label0 += 1
        new_label0 += 1

    labelNames = [0,1,2,3]
    train_set = {'data': X_train, 'labels': y_train, 'person': person_train}
    test_set = {'data': X_test, 'labels': y_test, 'person': person_test}
    val_set = {'data': X_val, 'labels': y_val, 'person': person_val}
    return train_set, test_set, val_set, labelNames


def sample_batch(dataset, batch_size):
    N = dataset['data'].shape[0]
    indices = np.random.randint(N, size=batch_size)
    return {key: dataset[key][indices] for key in dataset}

def train_model_ss(target_dir,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss,num_class,max_step,log_frequency,batch_size,model_saving_freq):
    # load data
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader_ss(time_length)
    print("Dataset loading completes.")
    
    best_val_acc = 0
    best_val_path = target_dir
    


    # reset default graph
    tf.reset_default_graph()

    global_step = tf.train.get_or_create_global_step()
    
    model = Model(config, labelNames,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)

    # training setups
    saver = tf.train.Saver(max_to_keep=100)

    # Build an initialization operation to run below.
    init = tf.global_variables_initializer()
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off
    
    sess = tf.Session(config=config_proto)

    sess.run(init)

    # main loop
    for step in range(max_step):
        start_time = time.time()

        # prepare data
        train_batch = sample_batch(trainSet, batch_size)
        # feed dict
        feed_dict = {
            model.input: train_batch['data'],
            model.fine_labels: train_batch['labels'],
            model.is_training: True,
        }
        
        fetch_list = [model.optimizer_op, model.loss, model.global_step]
        _, loss_value, global_step_value = sess.run(fetch_list, feed_dict=feed_dict)
        duration = time.time() - start_time
        assert not np.isnan(loss_value), 'Model diverged with loss = NaN'

        # log
        if step % log_frequency == 0:
            num_examples_per_step = batch_size
            examples_per_sec = num_examples_per_step / duration
            sec_per_batch = duration
            format_str = (
                '%s: step %d, examples %d, loss = %.9f (%.3f examples/sec; %.3f sec/batch)'
            )
            print(
                format_str % (
                    datetime.now(), step, batch_size * step,
                    loss_value,
                    examples_per_sec, sec_per_batch
                )
            )

        # Save the model checkpoint periodically.
        if step % model_saving_freq == 0 or (step + 1) == max_step:
            fetch_list = [model.accuracy,model.top_2_accuracy]
            feed_dict = {
                model.input: valSet['data'],
                model.fine_labels: valSet['labels'],
                model.is_training: False,
            }
            val_acc_1, val_acc_2 =  sess.run(fetch_list, feed_dict=feed_dict)
            checkpoint_path = os.path.join(target_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step=int(global_step_value))
            print('Top 1 validation accuracy: {} and top 2 validation accuracy: {}'.format(val_acc_1,val_acc_2))
            print('Model Saved!')
            if val_acc_1>best_val_acc:
                best_val_acc = val_acc_1
                best_val_path = checkpoint_path+'-'+str(int(global_step_value))
    return best_val_acc,best_val_path


In [46]:
best_val_ss = {}

num_cls = 4
mstp = 5000
lfrq = 10
bsz = 50
msf = 100
tr = './trained_model_final/DCNN_reg_ss'
if not os.path.exists(tr):
    os.mkdir(tr)
start = 0
stop = 350
step = 100
time_length = 700
time_bin = 350
fsz = 6
use_batchnorm = True
use_dropout = True
use_l2loss = True
acc = 'Accuracy'
path = 'Path'

best_val_ss[acc],best_val_ss[path] = train_model_ss(tr,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss,num_cls,mstp,lfrq,bsz,msf)

Loading datasets...
Dataset loading completes.
2019-03-16 12:41:42.203048: step 0, examples 0, loss = 1.439974308 (98.401 examples/sec; 0.508 sec/batch)
Top 1 validation accuracy: 0.2688679099082947 and top 2 validation accuracy: 0.5023584961891174
Model Saved!
2019-03-16 12:41:45.733314: step 10, examples 500, loss = 1.443587065 (283.336 examples/sec; 0.176 sec/batch)
2019-03-16 12:41:47.509036: step 20, examples 1000, loss = 1.419790626 (284.954 examples/sec; 0.175 sec/batch)
2019-03-16 12:41:49.305814: step 30, examples 1500, loss = 1.413360476 (280.152 examples/sec; 0.178 sec/batch)
2019-03-16 12:41:51.106602: step 40, examples 2000, loss = 1.400026321 (278.587 examples/sec; 0.179 sec/batch)
2019-03-16 12:41:52.965545: step 50, examples 2500, loss = 1.462016463 (266.669 examples/sec; 0.187 sec/batch)
2019-03-16 12:41:54.817471: step 60, examples 3000, loss = 1.342422605 (271.015 examples/sec; 0.184 sec/batch)
2019-03-16 12:41:56.667389: step 70, examples 3500, loss = 1.303992629 (2

2019-03-16 12:43:55.743025: step 660, examples 33000, loss = 0.685092866 (271.017 examples/sec; 0.184 sec/batch)
2019-03-16 12:43:57.595952: step 670, examples 33500, loss = 0.965965927 (275.509 examples/sec; 0.181 sec/batch)
2019-03-16 12:43:59.435844: step 680, examples 34000, loss = 0.798896730 (275.509 examples/sec; 0.181 sec/batch)
2019-03-16 12:44:01.301806: step 690, examples 34500, loss = 0.703344345 (261.084 examples/sec; 0.192 sec/batch)
2019-03-16 12:44:03.132674: step 700, examples 35000, loss = 1.078117490 (272.498 examples/sec; 0.183 sec/batch)
Top 1 validation accuracy: 0.5660377144813538 and top 2 validation accuracy: 0.7971698045730591
Model Saved!
2019-03-16 12:44:06.561793: step 710, examples 35500, loss = 0.700240493 (272.498 examples/sec; 0.183 sec/batch)
2019-03-16 12:44:08.409707: step 720, examples 36000, loss = 0.714880288 (273.995 examples/sec; 0.182 sec/batch)
2019-03-16 12:44:10.257620: step 730, examples 36500, loss = 0.706665397 (269.552 examples/sec; 0.18

2019-03-16 12:46:11.527280: step 1320, examples 66000, loss = 0.447199225 (275.509 examples/sec; 0.181 sec/batch)
2019-03-16 12:46:13.427332: step 1330, examples 66500, loss = 0.658398032 (266.668 examples/sec; 0.187 sec/batch)
2019-03-16 12:46:15.302318: step 1340, examples 67000, loss = 0.500437260 (266.669 examples/sec; 0.187 sec/batch)
2019-03-16 12:46:17.165272: step 1350, examples 67500, loss = 0.846802354 (258.378 examples/sec; 0.194 sec/batch)
2019-03-16 12:46:19.038252: step 1360, examples 68000, loss = 0.541093230 (273.995 examples/sec; 0.182 sec/batch)
2019-03-16 12:46:20.861099: step 1370, examples 68500, loss = 0.561710835 (275.509 examples/sec; 0.181 sec/batch)
2019-03-16 12:46:22.888490: step 1380, examples 69000, loss = 0.727548540 (261.084 examples/sec; 0.192 sec/batch)
2019-03-16 12:46:24.767487: step 1390, examples 69500, loss = 0.716202557 (263.845 examples/sec; 0.190 sec/batch)
2019-03-16 12:46:26.670547: step 1400, examples 70000, loss = 0.450971544 (275.509 examp

2019-03-16 12:48:27.525915: step 1990, examples 99500, loss = 0.393997908 (263.847 examples/sec; 0.190 sec/batch)
2019-03-16 12:48:29.433990: step 2000, examples 100000, loss = 0.550537050 (271.015 examples/sec; 0.184 sec/batch)
Top 1 validation accuracy: 0.6108490824699402 and top 2 validation accuracy: 0.8349056839942932
Model Saved!
2019-03-16 12:48:32.907225: step 2010, examples 100500, loss = 0.892129362 (269.552 examples/sec; 0.185 sec/batch)
2019-03-16 12:48:34.774189: step 2020, examples 101000, loss = 0.551193595 (273.995 examples/sec; 0.182 sec/batch)
2019-03-16 12:48:36.692290: step 2030, examples 101500, loss = 0.449397773 (268.102 examples/sec; 0.186 sec/batch)
2019-03-16 12:48:38.570283: step 2040, examples 102000, loss = 0.603187680 (275.509 examples/sec; 0.181 sec/batch)
2019-03-16 12:48:40.433237: step 2050, examples 102500, loss = 0.461228192 (265.249 examples/sec; 0.189 sec/batch)
2019-03-16 12:48:42.317248: step 2060, examples 103000, loss = 0.403281450 (257.046 exa

2019-03-16 12:50:40.233309: step 2640, examples 132000, loss = 0.511146903 (280.152 examples/sec; 0.178 sec/batch)
2019-03-16 12:50:42.096979: step 2650, examples 132500, loss = 0.314398587 (260.111 examples/sec; 0.192 sec/batch)
2019-03-16 12:50:44.128746: step 2660, examples 133000, loss = 0.295097709 (231.942 examples/sec; 0.216 sec/batch)
2019-03-16 12:50:46.082943: step 2670, examples 133500, loss = 0.396839589 (261.084 examples/sec; 0.192 sec/batch)
2019-03-16 12:50:48.113341: step 2680, examples 134000, loss = 0.448509991 (255.728 examples/sec; 0.196 sec/batch)
2019-03-16 12:50:50.092604: step 2690, examples 134500, loss = 0.304269105 (258.379 examples/sec; 0.194 sec/batch)
2019-03-16 12:50:52.092964: step 2700, examples 135000, loss = 0.404994667 (266.669 examples/sec; 0.187 sec/batch)
Top 1 validation accuracy: 0.5872641801834106 and top 2 validation accuracy: 0.8136792182922363
Model Saved!
2019-03-16 12:50:55.674488: step 2710, examples 135500, loss = 0.427383304 (275.507 ex

2019-03-16 12:52:56.934255: step 3300, examples 165000, loss = 0.366421789 (266.638 examples/sec; 0.188 sec/batch)
Top 1 validation accuracy: 0.5872641801834106 and top 2 validation accuracy: 0.801886796951294
Model Saved!
2019-03-16 12:53:00.404934: step 3310, examples 165500, loss = 0.380724847 (265.098 examples/sec; 0.189 sec/batch)
2019-03-16 12:53:02.253592: step 3320, examples 166000, loss = 0.283658832 (289.126 examples/sec; 0.173 sec/batch)
2019-03-16 12:53:04.123012: step 3330, examples 166500, loss = 0.306563020 (266.621 examples/sec; 0.188 sec/batch)
2019-03-16 12:53:05.983651: step 3340, examples 167000, loss = 0.633226991 (266.639 examples/sec; 0.188 sec/batch)
2019-03-16 12:53:07.871726: step 3350, examples 167500, loss = 0.501171350 (266.638 examples/sec; 0.188 sec/batch)
2019-03-16 12:53:09.747175: step 3360, examples 168000, loss = 0.468441784 (290.876 examples/sec; 0.172 sec/batch)
2019-03-16 12:53:11.622806: step 3370, examples 168500, loss = 0.606741250 (287.227 exa

2019-03-16 12:55:09.684208: step 3950, examples 197500, loss = 0.364966094 (301.671 examples/sec; 0.166 sec/batch)
2019-03-16 12:55:11.534193: step 3960, examples 198000, loss = 0.444897592 (261.758 examples/sec; 0.191 sec/batch)
2019-03-16 12:55:13.358503: step 3970, examples 198500, loss = 0.320079654 (263.539 examples/sec; 0.190 sec/batch)
2019-03-16 12:55:15.250411: step 3980, examples 199000, loss = 0.406709969 (255.728 examples/sec; 0.196 sec/batch)
2019-03-16 12:55:17.222669: step 3990, examples 199500, loss = 0.467290461 (265.249 examples/sec; 0.189 sec/batch)
2019-03-16 12:55:19.128094: step 4000, examples 200000, loss = 0.354816914 (260.594 examples/sec; 0.192 sec/batch)
Top 1 validation accuracy: 0.6226415038108826 and top 2 validation accuracy: 0.8254716992378235
Model Saved!
2019-03-16 12:55:22.709941: step 4010, examples 200500, loss = 0.387349308 (269.552 examples/sec; 0.185 sec/batch)
2019-03-16 12:55:24.647594: step 4020, examples 201000, loss = 0.453674495 (273.244 ex

Top 1 validation accuracy: 0.5707547068595886 and top 2 validation accuracy: 0.8089622855186462
Model Saved!
2019-03-16 12:57:28.451822: step 4610, examples 230500, loss = 0.321591407 (277.686 examples/sec; 0.180 sec/batch)
2019-03-16 12:57:30.546657: step 4620, examples 231000, loss = 0.421681494 (184.693 examples/sec; 0.271 sec/batch)
2019-03-16 12:57:33.119345: step 4630, examples 231500, loss = 0.468222260 (158.308 examples/sec; 0.316 sec/batch)
2019-03-16 12:57:35.253913: step 4640, examples 232000, loss = 0.351315707 (270.844 examples/sec; 0.185 sec/batch)
2019-03-16 12:57:37.107156: step 4650, examples 232500, loss = 0.359872371 (264.833 examples/sec; 0.189 sec/batch)
2019-03-16 12:57:39.200004: step 4660, examples 233000, loss = 0.298920274 (259.955 examples/sec; 0.192 sec/batch)
2019-03-16 12:57:41.246855: step 4670, examples 233500, loss = 0.334362924 (244.446 examples/sec; 0.205 sec/batch)
2019-03-16 12:57:43.202566: step 4680, examples 234000, loss = 0.294347644 (239.745 ex

In [47]:

def test_model_ss(model_dir,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss):
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader_ss(time_length)
    print("Dataset loading completes.")
    
    # reset default graph
    tf.reset_default_graph()
    
    # create model
    
    model = Model(config, labelNames,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)
    # training setups
    saver = tf.train.Saver(max_to_keep=100)
    test_accuracy = [0, 0]
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off

    with tf.Session(config=config_proto) as session:
        session.run(tf.global_variables_initializer())
        saver.restore(session, model_dir)
        # test model
        exp_results = run_single_step(session, model, testSet, mode='test')
        test_accuracy[0] = exp_results['top_1_accuracy']
        test_accuracy[1] = exp_results['top_2_accuracy']
        print('top_1_accuracy_test = ', test_accuracy[0], 'top_2_accuracy_test = ', test_accuracy[1])
        
    top1_acc = test_accuracy[0]
    top2_acc = test_accuracy[1]
    return top1_acc,top2_acc


def run_single_step(
        session,
        model,
        batch,
        mode='test',
        log=True,
):
    # construct feed dict
    feed_dict = {
        model.input: batch['data'],
        # model.coarse_labels: batch['coarse_labels'],
        model.fine_labels: batch['labels'],
        # model.label_mapping: label_mapping,
        model.is_training: mode == 'train'
    }
    
    # select proper summary op
    if mode == 'train':
        summary_op = model.train_summary_op
    elif mode == 'val':
        summary_op = model.val_summary_op
    else:
        summary_op = model.test_summary_op
    
    # construct fetch list
    fetch_list = [model.global_step, summary_op, model.loss, model.accuracy, model.top_2_accuracy]

    # run single step
    _start_time = time.time()
    _step, _summary, _loss, _top_1, _top_2 = session.run(fetch_list, feed_dict=feed_dict)[:5]
    _end_time = time.time()
    
    # collect step statistics
    step_time = _end_time - _start_time
    batch_size = batch['data'].shape[0]
    
    # log in console
    if log:
        print(('[{:5s} step {:4d}] loss: {:.5f}; top_1_accuracy: {:.5f}; top_5_accuracy: {:5f} ' +
              '({:.3f} sec/batch; {:.3f} instances/sec)'
              ).format(mode, _step, _loss, _top_1, _top_2, 
                       step_time, batch_size / step_time))
    
    # log results to file and return statistics
    if mode == 'test':
        test_fetch_list = [model.per_class_accuracy,
                model.top_2_per_class_accuracy,
                model.confusion_matrix, 
                model.pred, model.probs]
        _top_1_c,  _top_2_c, _cm, _pred, _probs = \
                session.run(test_fetch_list, feed_dict=feed_dict)
        
        
        # Log detailed test results in pickle format
        stats = {
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2,
            "top_1_perclass_accuracy": _top_1_c,
            "top_2_perclass_accuracy": _top_2_c,
            "confusion_matrix": _cm,
            "pred": _pred,
            "probs": _probs
        }
    else:
        stats = {
            "step": _step,
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2
        }
        
    return stats

In [51]:
# Accuracy for testSet(best)

path = 'Path'
model_dir = best_val_ss[path]
start = 0
stop = 350
step = 100
time_length = 700
time_bin = 350
#Nb = BATCH_SIZE
fsz = 6
use_batchnorm = True
use_dropout = True
use_l2loss = True
print('Test Accuracy for Subsampling:')
best_test_acc_ss, best_test_acc2_ss = test_model_ss(model_dir,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)

Test Accuracy for Subsampling:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss\model.ckpt-1601
[test  step 1601] loss: 1.43090; top_1_accuracy: 0.59594; top_5_accuracy: 0.811512 (1.050 sec/batch; 843.693 instances/sec)
top_1_accuracy_test =  0.5959368 top_2_accuracy_test =  0.8115124


In [54]:
#try with 1000 timelength

num_cls = 4
mstp = 5000
lfrq = 10
bsz = 100
msf = 100
tr = './trained_model_final/DCNN_reg_ss1000'
if not os.path.exists(tr):
    os.mkdir(tr)
start = 0
stop = 500
step = 100
time_length = 1000
time_bin = 500
fsz = 5
use_batchnorm = True
use_dropout = True
use_l2loss = True
acc = 'Accuracy1000'
path = 'Path1000'

best_val_ss[acc],best_val_ss[path] = train_model_ss(tr,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss,num_cls,mstp,lfrq,bsz,msf)

Loading datasets...
Dataset loading completes.
2019-03-16 16:40:52.871784: step 0, examples 0, loss = 1.441268325 (118.512 examples/sec; 0.844 sec/batch)
Top 1 validation accuracy: 0.25943395495414734 and top 2 validation accuracy: 0.5070754885673523
Model Saved!
2019-03-16 16:40:59.747225: step 10, examples 1000, loss = 1.427458048 (199.686 examples/sec; 0.501 sec/batch)
2019-03-16 16:41:04.706026: step 20, examples 2000, loss = 1.415421605 (197.539 examples/sec; 0.506 sec/batch)
2019-03-16 16:41:09.684045: step 30, examples 3000, loss = 1.361649752 (206.474 examples/sec; 0.484 sec/batch)
2019-03-16 16:41:14.622343: step 40, examples 4000, loss = 1.359479785 (193.929 examples/sec; 0.516 sec/batch)
2019-03-16 16:41:19.559762: step 50, examples 5000, loss = 1.280585289 (212.384 examples/sec; 0.471 sec/batch)
2019-03-16 16:41:24.605970: step 60, examples 6000, loss = 1.238089681 (190.103 examples/sec; 0.526 sec/batch)
2019-03-16 16:41:29.498151: step 70, examples 7000, loss = 1.238902211

2019-03-16 16:46:39.939083: step 660, examples 66000, loss = 0.400830775 (200.779 examples/sec; 0.498 sec/batch)
2019-03-16 16:46:44.983823: step 670, examples 67000, loss = 0.538155675 (199.613 examples/sec; 0.501 sec/batch)
2019-03-16 16:46:49.994361: step 680, examples 68000, loss = 0.662711024 (201.462 examples/sec; 0.496 sec/batch)
2019-03-16 16:46:55.074607: step 690, examples 69000, loss = 0.441506356 (201.759 examples/sec; 0.496 sec/batch)
2019-03-16 16:47:00.199999: step 700, examples 70000, loss = 0.588956058 (193.961 examples/sec; 0.516 sec/batch)
Top 1 validation accuracy: 0.5542452931404114 and top 2 validation accuracy: 0.7783018946647644
Model Saved!
2019-03-16 16:47:06.950417: step 710, examples 71000, loss = 0.549572229 (200.455 examples/sec; 0.499 sec/batch)
2019-03-16 16:47:11.966814: step 720, examples 72000, loss = 0.552448273 (194.203 examples/sec; 0.515 sec/batch)
2019-03-16 16:47:16.974275: step 730, examples 73000, loss = 0.557506084 (191.334 examples/sec; 0.52

2019-03-16 16:52:32.742864: step 1320, examples 132000, loss = 0.313344002 (201.779 examples/sec; 0.496 sec/batch)
2019-03-16 16:52:38.062218: step 1330, examples 133000, loss = 0.311183989 (177.454 examples/sec; 0.564 sec/batch)
2019-03-16 16:52:43.314601: step 1340, examples 134000, loss = 0.417152643 (187.107 examples/sec; 0.534 sec/batch)
2019-03-16 16:52:48.583555: step 1350, examples 135000, loss = 0.435539961 (190.942 examples/sec; 0.524 sec/batch)
2019-03-16 16:52:53.923902: step 1360, examples 136000, loss = 0.398967862 (192.928 examples/sec; 0.518 sec/batch)
2019-03-16 16:52:59.268416: step 1370, examples 137000, loss = 0.349134684 (186.048 examples/sec; 0.537 sec/batch)
2019-03-16 16:53:04.482928: step 1380, examples 138000, loss = 0.325918943 (191.649 examples/sec; 0.522 sec/batch)
2019-03-16 16:53:10.309114: step 1390, examples 139000, loss = 0.289816320 (171.512 examples/sec; 0.583 sec/batch)
2019-03-16 16:53:16.222372: step 1400, examples 140000, loss = 0.405950606 (160.

2019-03-16 16:58:29.684326: step 1980, examples 198000, loss = 0.367703974 (173.173 examples/sec; 0.577 sec/batch)
2019-03-16 16:58:34.939577: step 1990, examples 199000, loss = 0.381784260 (202.112 examples/sec; 0.495 sec/batch)
2019-03-16 16:58:39.978063: step 2000, examples 200000, loss = 0.403036058 (190.007 examples/sec; 0.526 sec/batch)
Top 1 validation accuracy: 0.5683962106704712 and top 2 validation accuracy: 0.7712264060974121
Model Saved!
2019-03-16 16:58:46.927504: step 2010, examples 201000, loss = 0.379186749 (191.642 examples/sec; 0.522 sec/batch)
2019-03-16 16:58:51.926740: step 2020, examples 202000, loss = 0.411801457 (210.640 examples/sec; 0.475 sec/batch)
2019-03-16 16:58:56.952939: step 2030, examples 203000, loss = 0.345443606 (199.592 examples/sec; 0.501 sec/batch)
2019-03-16 16:59:02.059290: step 2040, examples 204000, loss = 0.236854076 (199.891 examples/sec; 0.500 sec/batch)
2019-03-16 16:59:07.239929: step 2050, examples 205000, loss = 0.352337897 (197.307 ex

2019-03-16 17:04:12.324227: step 2630, examples 263000, loss = 0.262681931 (201.536 examples/sec; 0.496 sec/batch)
2019-03-16 17:04:17.300072: step 2640, examples 264000, loss = 0.348822594 (203.810 examples/sec; 0.491 sec/batch)
2019-03-16 17:04:22.342773: step 2650, examples 265000, loss = 0.385422468 (205.682 examples/sec; 0.486 sec/batch)
2019-03-16 17:04:27.451437: step 2660, examples 266000, loss = 0.266425759 (208.494 examples/sec; 0.480 sec/batch)
2019-03-16 17:04:32.451347: step 2670, examples 267000, loss = 0.274459481 (200.688 examples/sec; 0.498 sec/batch)
2019-03-16 17:04:37.536450: step 2680, examples 268000, loss = 0.493863881 (198.715 examples/sec; 0.503 sec/batch)
2019-03-16 17:04:42.574818: step 2690, examples 269000, loss = 0.347118676 (194.408 examples/sec; 0.514 sec/batch)
2019-03-16 17:04:47.653349: step 2700, examples 270000, loss = 0.264167756 (199.734 examples/sec; 0.501 sec/batch)
Top 1 validation accuracy: 0.5400943160057068 and top 2 validation accuracy: 0.7

2019-03-16 17:09:59.819539: step 3290, examples 329000, loss = 0.343237758 (207.619 examples/sec; 0.482 sec/batch)
2019-03-16 17:10:04.856148: step 3300, examples 330000, loss = 0.361011028 (194.695 examples/sec; 0.514 sec/batch)
Top 1 validation accuracy: 0.5660377144813538 and top 2 validation accuracy: 0.7688679099082947
Model Saved!
2019-03-16 17:10:11.903602: step 3310, examples 331000, loss = 0.226317808 (194.438 examples/sec; 0.514 sec/batch)
2019-03-16 17:10:17.299918: step 3320, examples 332000, loss = 0.341535330 (187.588 examples/sec; 0.533 sec/batch)
2019-03-16 17:10:22.607448: step 3330, examples 333000, loss = 0.250280052 (182.226 examples/sec; 0.549 sec/batch)
2019-03-16 17:10:27.925445: step 3340, examples 334000, loss = 0.354892343 (186.569 examples/sec; 0.536 sec/batch)
2019-03-16 17:10:33.200091: step 3350, examples 335000, loss = 0.364586174 (188.276 examples/sec; 0.531 sec/batch)
2019-03-16 17:10:38.405287: step 3360, examples 336000, loss = 0.231449530 (187.577 ex

2019-03-16 17:15:49.498319: step 3940, examples 394000, loss = 0.357687175 (202.441 examples/sec; 0.494 sec/batch)
2019-03-16 17:15:54.498604: step 3950, examples 395000, loss = 0.275132984 (205.031 examples/sec; 0.488 sec/batch)
2019-03-16 17:15:59.405693: step 3960, examples 396000, loss = 0.391265929 (200.171 examples/sec; 0.500 sec/batch)
2019-03-16 17:16:04.684750: step 3970, examples 397000, loss = 0.302908003 (199.560 examples/sec; 0.501 sec/batch)
2019-03-16 17:16:09.747937: step 3980, examples 398000, loss = 0.297354519 (198.604 examples/sec; 0.504 sec/batch)
2019-03-16 17:16:14.866790: step 3990, examples 399000, loss = 0.220366731 (186.068 examples/sec; 0.537 sec/batch)
2019-03-16 17:16:19.934811: step 4000, examples 400000, loss = 0.315451384 (205.787 examples/sec; 0.486 sec/batch)
Top 1 validation accuracy: 0.5589622855186462 and top 2 validation accuracy: 0.7948113083839417
Model Saved!
2019-03-16 17:16:26.888161: step 4010, examples 401000, loss = 0.401183337 (188.743 ex

2019-03-16 17:21:35.747137: step 4600, examples 460000, loss = 0.279576302 (198.916 examples/sec; 0.503 sec/batch)
Top 1 validation accuracy: 0.6014150977134705 and top 2 validation accuracy: 0.823113203048706
Model Saved!
2019-03-16 17:21:42.673799: step 4610, examples 461000, loss = 0.252703637 (197.815 examples/sec; 0.506 sec/batch)
2019-03-16 17:21:47.792394: step 4620, examples 462000, loss = 0.239071518 (191.090 examples/sec; 0.523 sec/batch)
2019-03-16 17:21:52.778127: step 4630, examples 463000, loss = 0.229563385 (198.926 examples/sec; 0.503 sec/batch)
2019-03-16 17:21:57.856753: step 4640, examples 464000, loss = 0.304680109 (207.084 examples/sec; 0.483 sec/batch)
2019-03-16 17:22:02.860971: step 4650, examples 465000, loss = 0.293845713 (205.443 examples/sec; 0.487 sec/batch)
2019-03-16 17:22:07.945684: step 4660, examples 466000, loss = 0.341687858 (202.337 examples/sec; 0.494 sec/batch)
2019-03-16 17:22:12.984706: step 4670, examples 467000, loss = 0.371105433 (199.092 exa

In [55]:
# Accuracy for testSet(best)

path = 'Path1000'
model_dir = best_val_ss[path]
start = 0
stop = 500
step = 100
time_length = 1000
time_bin = 500
#Nb = BATCH_SIZE
fsz = 5
use_batchnorm = True
use_dropout = True
use_l2loss = True
print('Test Accuracy for Subsampling:')
best_test_acc_ss1000, best_test_acc2_ss1000 = test_model_ss(model_dir,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)

Test Accuracy for Subsampling:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss1000\model.ckpt-3101
[test  step 3101] loss: 2.22320; top_1_accuracy: 0.56772; top_5_accuracy: 0.802483 (1.451 sec/batch; 610.616 instances/sec)
top_1_accuracy_test =  0.5677201 top_2_accuracy_test =  0.8024831


In [58]:
best_val_ss_crop = {}

for step_size in range(25,101,25):
    num_cls = 4
    mstp = 5000
    lfrq = 10
    bsz = 50
    msf = 100
    tr = './trained_model_final/DCNN_reg_ss_crop_step'+str(step_size)
    if not os.path.exists(tr):
        os.mkdir(tr)
    start = 0
    stop = 250
    step = step_size
    time_length = 700
    time_bin = 350
    fsz = 3
    use_batchnorm = True
    use_dropout = True
    use_l2loss = True
    acc = 'Accuracy_step'+str(step_size)
    path = 'Path_step'+str(step_size)

    best_val_ss_crop[acc],best_val_ss_crop[path] = train_model_ss(tr,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss,num_cls,mstp,lfrq,bsz,msf)

Loading datasets...
Dataset loading completes.
2019-03-16 21:57:08.009881: step 0, examples 0, loss = 1.419823050 (51.416 examples/sec; 0.972 sec/batch)
Top 1 validation accuracy: 0.2641509473323822 and top 2 validation accuracy: 0.5094339847564697
Model Saved!
2019-03-16 21:57:17.371099: step 10, examples 500, loss = 1.422534585 (77.302 examples/sec; 0.647 sec/batch)
2019-03-16 21:57:23.702835: step 20, examples 1000, loss = 1.412981749 (81.505 examples/sec; 0.613 sec/batch)
2019-03-16 21:57:30.084881: step 30, examples 1500, loss = 1.397699714 (78.495 examples/sec; 0.637 sec/batch)
2019-03-16 21:57:36.608701: step 40, examples 2000, loss = 1.410016060 (75.445 examples/sec; 0.663 sec/batch)
2019-03-16 21:57:43.051827: step 50, examples 2500, loss = 1.431589603 (80.750 examples/sec; 0.619 sec/batch)
2019-03-16 21:57:49.555563: step 60, examples 3000, loss = 1.396692991 (78.712 examples/sec; 0.635 sec/batch)
2019-03-16 21:57:56.184767: step 70, examples 3500, loss = 1.369373441 (75.897 

2019-03-16 22:04:47.812763: step 670, examples 33500, loss = 0.948702991 (73.864 examples/sec; 0.677 sec/batch)
2019-03-16 22:04:54.643132: step 680, examples 34000, loss = 1.092903852 (73.764 examples/sec; 0.678 sec/batch)
2019-03-16 22:05:01.356340: step 690, examples 34500, loss = 1.193002820 (77.178 examples/sec; 0.648 sec/batch)
2019-03-16 22:05:08.200232: step 700, examples 35000, loss = 0.961881459 (75.057 examples/sec; 0.666 sec/batch)
Top 1 validation accuracy: 0.5377358198165894 and top 2 validation accuracy: 0.75
Model Saved!
2019-03-16 22:05:17.844918: step 710, examples 35500, loss = 1.169107318 (71.569 examples/sec; 0.699 sec/batch)
2019-03-16 22:05:24.637679: step 720, examples 36000, loss = 1.106561661 (73.668 examples/sec; 0.679 sec/batch)
2019-03-16 22:05:31.422005: step 730, examples 36500, loss = 1.093246102 (75.349 examples/sec; 0.664 sec/batch)
2019-03-16 22:05:38.293704: step 740, examples 37000, loss = 1.204229355 (72.109 examples/sec; 0.693 sec/batch)
2019-03-1

2019-03-16 22:12:44.120162: step 1340, examples 67000, loss = 0.961541951 (73.119 examples/sec; 0.684 sec/batch)
2019-03-16 22:12:51.175450: step 1350, examples 67500, loss = 0.903099060 (70.285 examples/sec; 0.711 sec/batch)
2019-03-16 22:12:57.902270: step 1360, examples 68000, loss = 0.995333314 (75.311 examples/sec; 0.664 sec/batch)
2019-03-16 22:13:04.689359: step 1370, examples 68500, loss = 0.909675241 (73.577 examples/sec; 0.680 sec/batch)
2019-03-16 22:13:11.367693: step 1380, examples 69000, loss = 0.830238819 (76.310 examples/sec; 0.655 sec/batch)
2019-03-16 22:13:18.227918: step 1390, examples 69500, loss = 0.928257167 (69.815 examples/sec; 0.716 sec/batch)
2019-03-16 22:13:25.165313: step 1400, examples 70000, loss = 1.003360987 (72.842 examples/sec; 0.686 sec/batch)
Top 1 validation accuracy: 0.5896226167678833 and top 2 validation accuracy: 0.801886796951294
Model Saved!
2019-03-16 22:13:35.253788: step 1410, examples 70500, loss = 1.104883194 (71.389 examples/sec; 0.700

Top 1 validation accuracy: 0.5966981053352356 and top 2 validation accuracy: 0.7971698045730591
Model Saved!
2019-03-16 22:20:38.494558: step 2010, examples 100500, loss = 1.051523685 (78.142 examples/sec; 0.640 sec/batch)
2019-03-16 22:20:45.175909: step 2020, examples 101000, loss = 0.924017310 (71.518 examples/sec; 0.699 sec/batch)
2019-03-16 22:20:51.991012: step 2030, examples 101500, loss = 0.968584657 (74.791 examples/sec; 0.669 sec/batch)
2019-03-16 22:20:58.686104: step 2040, examples 102000, loss = 0.935776532 (74.886 examples/sec; 0.668 sec/batch)
2019-03-16 22:21:05.460909: step 2050, examples 102500, loss = 0.833380044 (73.913 examples/sec; 0.676 sec/batch)
2019-03-16 22:21:12.162680: step 2060, examples 103000, loss = 0.869185805 (73.508 examples/sec; 0.680 sec/batch)
2019-03-16 22:21:18.879312: step 2070, examples 103500, loss = 0.787920117 (73.474 examples/sec; 0.681 sec/batch)
2019-03-16 22:21:25.630560: step 2080, examples 104000, loss = 0.963711560 (74.853 examples/s

2019-03-16 22:28:23.876192: step 2670, examples 133500, loss = 0.695736706 (73.246 examples/sec; 0.683 sec/batch)
2019-03-16 22:28:30.563864: step 2680, examples 134000, loss = 0.746519148 (74.382 examples/sec; 0.672 sec/batch)
2019-03-16 22:28:37.335088: step 2690, examples 134500, loss = 0.681845725 (75.102 examples/sec; 0.666 sec/batch)
2019-03-16 22:28:44.074336: step 2700, examples 135000, loss = 0.754707694 (72.615 examples/sec; 0.689 sec/batch)
Top 1 validation accuracy: 0.6132075190544128 and top 2 validation accuracy: 0.8372641801834106
Model Saved!
2019-03-16 22:28:53.758897: step 2710, examples 135500, loss = 0.854095280 (75.679 examples/sec; 0.661 sec/batch)
2019-03-16 22:29:00.516162: step 2720, examples 136000, loss = 0.817437947 (72.116 examples/sec; 0.693 sec/batch)
2019-03-16 22:29:07.324472: step 2730, examples 136500, loss = 0.729559958 (72.296 examples/sec; 0.692 sec/batch)
2019-03-16 22:29:13.992748: step 2740, examples 137000, loss = 0.972869694 (76.135 examples/s

...

2019-03-17 00:02:21.996940: step 2150, examples 107500, loss = 0.978056431 (199.978 examples/sec; 0.250 sec/batch)
2019-03-17 00:02:24.537595: step 2160, examples 108000, loss = 1.018062592 (198.036 examples/sec; 0.252 sec/batch)
2019-03-17 00:02:27.033224: step 2170, examples 108500, loss = 0.738855720 (199.978 examples/sec; 0.250 sec/batch)
2019-03-17 00:02:29.514192: step 2180, examples 109000, loss = 1.068460226 (193.628 examples/sec; 0.258 sec/batch)
2019-03-17 00:02:31.965799: step 2190, examples 109500, loss = 1.008248210 (213.307 examples/sec; 0.234 sec/batch)
2019-03-17 00:02:34.442388: step 2200, examples 110000, loss = 0.807053566 (201.164 examples/sec; 0.249 sec/batch)
Top 1 validation accuracy: 0.5566037893295288 and top 2 validation accuracy: 0.75
Model Saved!
2019-03-17 00:02:38.683300: step 2210, examples 110500, loss = 0.614175498 (209.913 examples/sec; 0.238 sec/batch)
2019-03-17 00:02:41.210165: step 2220, examples 111000, loss = 0.989982665 (187.598 examples/sec; 0.

Top 1 validation accuracy: 0.5566037893295288 and top 2 validation accuracy: 0.7924528121948242
Model Saved!
2019-03-17 00:05:22.858374: step 2810, examples 140500, loss = 0.941961408 (174.972 examples/sec; 0.286 sec/batch)
2019-03-17 00:05:25.624731: step 2820, examples 141000, loss = 0.748861551 (166.223 examples/sec; 0.301 sec/batch)
2019-03-17 00:05:28.639747: step 2830, examples 141500, loss = 0.720973134 (171.956 examples/sec; 0.291 sec/batch)
2019-03-17 00:05:31.488322: step 2840, examples 142000, loss = 0.926611543 (169.616 examples/sec; 0.295 sec/batch)
2019-03-17 00:05:34.884353: step 2850, examples 142500, loss = 0.920786977 (134.413 examples/sec; 0.372 sec/batch)
2019-03-17 00:05:38.169088: step 2860, examples 143000, loss = 0.888668299 (132.625 examples/sec; 0.377 sec/batch)
2019-03-17 00:05:41.228222: step 2870, examples 143500, loss = 0.732246757 (152.499 examples/sec; 0.328 sec/batch)
2019-03-17 00:05:44.060754: step 2880, examples 144000, loss = 0.927811742 (176.208 ex

2019-03-17 00:08:37.952247: step 3460, examples 173000, loss = 1.042402864 (186.071 examples/sec; 0.269 sec/batch)
2019-03-17 00:08:40.622755: step 3470, examples 173500, loss = 0.904824018 (174.722 examples/sec; 0.286 sec/batch)
2019-03-17 00:08:43.329573: step 3480, examples 174000, loss = 0.773936033 (180.678 examples/sec; 0.277 sec/batch)
2019-03-17 00:08:46.311183: step 3490, examples 174500, loss = 0.793661714 (147.972 examples/sec; 0.338 sec/batch)
2019-03-17 00:08:49.556933: step 3500, examples 175000, loss = 0.915238738 (158.812 examples/sec; 0.315 sec/batch)
Top 1 validation accuracy: 0.5707547068595886 and top 2 validation accuracy: 0.8113207817077637
Model Saved!
2019-03-17 00:08:54.719854: step 3510, examples 175500, loss = 0.830311418 (174.360 examples/sec; 0.287 sec/batch)
2019-03-17 00:08:57.618562: step 3520, examples 176000, loss = 0.769705057 (185.035 examples/sec; 0.270 sec/batch)
2019-03-17 00:09:00.497718: step 3530, examples 176500, loss = 0.804015160 (173.450 ex

2019-03-17 00:11:40.478830: step 4110, examples 205500, loss = 0.933153152 (199.543 examples/sec; 0.251 sec/batch)
2019-03-17 00:11:43.029074: step 4120, examples 206000, loss = 0.742695093 (189.551 examples/sec; 0.264 sec/batch)
2019-03-17 00:11:45.709838: step 4130, examples 206500, loss = 0.691643238 (206.384 examples/sec; 0.242 sec/batch)
2019-03-17 00:11:48.298569: step 4140, examples 207000, loss = 0.639336944 (185.755 examples/sec; 0.269 sec/batch)
2019-03-17 00:11:51.100738: step 4150, examples 207500, loss = 0.783660948 (154.387 examples/sec; 0.324 sec/batch)
2019-03-17 00:11:53.897175: step 4160, examples 208000, loss = 0.762989163 (181.336 examples/sec; 0.276 sec/batch)
2019-03-17 00:11:56.594348: step 4170, examples 208500, loss = 0.785650373 (159.830 examples/sec; 0.313 sec/batch)
2019-03-17 00:11:59.327626: step 4180, examples 209000, loss = 0.782078862 (147.096 examples/sec; 0.340 sec/batch)
2019-03-17 00:12:02.262793: step 4190, examples 209500, loss = 0.781036437 (204.

2019-03-17 00:14:49.457837: step 4770, examples 238500, loss = 0.756168783 (162.964 examples/sec; 0.307 sec/batch)
2019-03-17 00:14:52.298390: step 4780, examples 239000, loss = 0.804214597 (181.335 examples/sec; 0.276 sec/batch)
2019-03-17 00:14:55.220159: step 4790, examples 239500, loss = 0.836674869 (174.972 examples/sec; 0.286 sec/batch)
2019-03-17 00:14:57.981502: step 4800, examples 240000, loss = 0.654715419 (181.334 examples/sec; 0.276 sec/batch)
Top 1 validation accuracy: 0.5400943160057068 and top 2 validation accuracy: 0.7900943160057068
Model Saved!
2019-03-17 00:15:02.529596: step 4810, examples 240500, loss = 0.980502546 (189.608 examples/sec; 0.264 sec/batch)
2019-03-17 00:15:05.133519: step 4820, examples 241000, loss = 0.643462896 (194.036 examples/sec; 0.258 sec/batch)
2019-03-17 00:15:07.733433: step 4830, examples 241500, loss = 0.714973569 (197.885 examples/sec; 0.253 sec/batch)
2019-03-17 00:15:10.400525: step 4840, examples 242000, loss = 0.660747528 (192.537 ex

In [59]:
# Accuracy for testSet(best)

best_test_acc_ss_crop = 0
best_test_ss_crop = 0
for step_size in range(25,101,25):
    path = 'Path_step'+str(step_size)
    model_dir = best_val_ss_crop[path]
    start = 0
    stop = 250
    step = step_size
    time_length = 700
    time_bin = 350
    #Nb = BATCH_SIZE
    fsz = 3
    use_batchnorm = True
    use_dropout = True
    use_l2loss = True
    print('Test Accuracy using cropped data for training for step size {}:'.format(step_size))
    top_1_acc_test, top_2_acc_test = test_model_ss(model_dir,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)
    if top_1_acc_test > best_test_acc_ss_crop:
        best_test_acc_ss_crop = top_1_acc_test
        best_test_ss_crop = step_size

print('The step_size {} has the best test accuracy: {}'.format(best_test_ss_crop,best_test_acc_ss_crop))

Test Accuracy using cropped data for training for step size 25:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_step25\model.ckpt-3501
[test  step 3501] loss: 1.27136; top_1_accuracy: 0.63883; top_5_accuracy: 0.844244 (3.742 sec/batch; 236.775 instances/sec)
top_1_accuracy_test =  0.6388262 top_2_accuracy_test =  0.84424376
Test Accuracy using cropped data for training for step size 50:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_step50\model.ckpt-4301
[test  step 4301] loss: 1.29987; top_1_accuracy: 0.62641; top_5_accuracy: 0.847630 (2.122 sec/batch; 417.606 instances/sec)
top_1_accuracy_test =  0.62641084 top_2_accuracy_test =  0.8476298
Test Accuracy using cropped data for training for step size 75:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_c

In [60]:
# Lets try also with step size 10
for step_size in range(10,11,10):
    num_cls = 4
    mstp = 5000
    lfrq = 10
    bsz = 50
    msf = 100
    tr = './trained_model_final/DCNN_reg_ss_crop_step'+str(step_size)
    if not os.path.exists(tr):
        os.mkdir(tr)
    start = 0
    stop = 250
    step = step_size
    time_length = 700
    time_bin = 350
    fsz = 3
    use_batchnorm = True
    use_dropout = True
    use_l2loss = True
    acc = 'Accuracy_step'+str(step_size)
    path = 'Path_step'+str(step_size)

    best_val_ss_crop[acc],best_val_ss_crop[path] = train_model_ss(tr,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss,num_cls,mstp,lfrq,bsz,msf)

Loading datasets...
Dataset loading completes.
2019-03-17 00:42:12.546478: step 0, examples 0, loss = 1.420341611 (26.219 examples/sec; 1.907 sec/batch)
Top 1 validation accuracy: 0.22641509771347046 and top 2 validation accuracy: 0.5
Model Saved!
2019-03-17 00:42:32.166555: step 10, examples 500, loss = 1.413772106 (33.512 examples/sec; 1.492 sec/batch)
2019-03-17 00:42:47.138594: step 20, examples 1000, loss = 1.405620575 (32.350 examples/sec; 1.546 sec/batch)
2019-03-17 00:43:02.152449: step 30, examples 1500, loss = 1.380291939 (33.591 examples/sec; 1.489 sec/batch)
2019-03-17 00:43:17.059271: step 40, examples 2000, loss = 1.335037708 (34.142 examples/sec; 1.464 sec/batch)
2019-03-17 00:43:31.894992: step 50, examples 2500, loss = 1.343952060 (34.417 examples/sec; 1.453 sec/batch)
2019-03-17 00:43:46.719845: step 60, examples 3000, loss = 1.401606202 (34.282 examples/sec; 1.458 sec/batch)
2019-03-17 00:44:01.770454: step 70, examples 3500, loss = 1.330043077 (33.146 examples/sec; 

2019-03-17 00:59:34.800604: step 670, examples 33500, loss = 1.122202158 (33.468 examples/sec; 1.494 sec/batch)
2019-03-17 00:59:50.532103: step 680, examples 34000, loss = 0.875912726 (33.831 examples/sec; 1.478 sec/batch)
2019-03-17 01:00:05.320426: step 690, examples 34500, loss = 1.025480509 (32.981 examples/sec; 1.516 sec/batch)
2019-03-17 01:00:20.477733: step 700, examples 35000, loss = 0.919461429 (33.334 examples/sec; 1.500 sec/batch)
Top 1 validation accuracy: 0.551886796951294 and top 2 validation accuracy: 0.7688679099082947
Model Saved!
2019-03-17 01:00:39.993626: step 710, examples 35500, loss = 1.060526848 (33.900 examples/sec; 1.475 sec/batch)
2019-03-17 01:00:54.931347: step 720, examples 36000, loss = 1.170174837 (33.558 examples/sec; 1.490 sec/batch)
2019-03-17 01:01:09.789858: step 730, examples 36500, loss = 1.015042186 (33.785 examples/sec; 1.480 sec/batch)
2019-03-17 01:01:24.681456: step 740, examples 37000, loss = 0.968021572 (33.178 examples/sec; 1.507 sec/bat

2019-03-17 01:16:47.531060: step 1340, examples 67000, loss = 1.064348221 (31.304 examples/sec; 1.597 sec/batch)
2019-03-17 01:17:03.195714: step 1350, examples 67500, loss = 0.792746007 (32.048 examples/sec; 1.560 sec/batch)
2019-03-17 01:17:18.825275: step 1360, examples 68000, loss = 0.871252000 (32.276 examples/sec; 1.549 sec/batch)
2019-03-17 01:17:34.353566: step 1370, examples 68500, loss = 0.758194983 (32.069 examples/sec; 1.559 sec/batch)
2019-03-17 01:17:49.943020: step 1380, examples 69000, loss = 0.916087210 (32.297 examples/sec; 1.548 sec/batch)
2019-03-17 01:18:05.570575: step 1390, examples 69500, loss = 1.002144098 (32.807 examples/sec; 1.524 sec/batch)
2019-03-17 01:18:20.999602: step 1400, examples 70000, loss = 0.743937731 (32.297 examples/sec; 1.548 sec/batch)
Top 1 validation accuracy: 0.6438679099082947 and top 2 validation accuracy: 0.8278301954269409
Model Saved!
2019-03-17 01:18:41.972372: step 1410, examples 70500, loss = 0.938104868 (31.762 examples/sec; 1.57

Top 1 validation accuracy: 0.6415094137191772 and top 2 validation accuracy: 0.823113203048706
Model Saved!
2019-03-17 01:34:47.732183: step 2010, examples 100500, loss = 0.954817951 (33.762 examples/sec; 1.481 sec/batch)
2019-03-17 01:35:03.020837: step 2020, examples 101000, loss = 0.787812948 (33.580 examples/sec; 1.489 sec/batch)
2019-03-17 01:35:18.292447: step 2030, examples 101500, loss = 0.808538258 (33.223 examples/sec; 1.505 sec/batch)
2019-03-17 01:35:33.578092: step 2040, examples 102000, loss = 0.853920341 (32.743 examples/sec; 1.527 sec/batch)
2019-03-17 01:35:48.893819: step 2050, examples 102500, loss = 0.755886972 (33.112 examples/sec; 1.510 sec/batch)
2019-03-17 01:36:04.415092: step 2060, examples 103000, loss = 0.978049994 (33.003 examples/sec; 1.515 sec/batch)
2019-03-17 01:36:19.714776: step 2070, examples 103500, loss = 0.815852225 (32.593 examples/sec; 1.534 sec/batch)
2019-03-17 01:36:35.114726: step 2080, examples 104000, loss = 0.801499128 (32.381 examples/se

2019-03-17 01:52:09.254738: step 2670, examples 133500, loss = 0.762892187 (31.642 examples/sec; 1.580 sec/batch)
2019-03-17 01:52:24.882294: step 2680, examples 134000, loss = 0.756363690 (31.946 examples/sec; 1.565 sec/batch)
2019-03-17 01:52:40.602094: step 2690, examples 134500, loss = 0.630979538 (32.007 examples/sec; 1.562 sec/batch)
2019-03-17 01:52:56.172498: step 2700, examples 135000, loss = 0.733646154 (32.721 examples/sec; 1.528 sec/batch)
Top 1 validation accuracy: 0.6155660152435303 and top 2 validation accuracy: 0.823113203048706
Model Saved!
2019-03-17 01:53:17.152285: step 2710, examples 135500, loss = 0.981679976 (31.128 examples/sec; 1.606 sec/batch)
2019-03-17 01:53:32.775830: step 2720, examples 136000, loss = 0.862564802 (32.829 examples/sec; 1.523 sec/batch)
2019-03-17 01:53:48.488613: step 2730, examples 136500, loss = 0.915446460 (31.844 examples/sec; 1.570 sec/batch)
2019-03-17 01:54:04.109149: step 2740, examples 137000, loss = 0.864117801 (32.700 examples/se

2019-03-17 02:09:48.905372: step 3330, examples 166500, loss = 0.740597486 (30.186 examples/sec; 1.656 sec/batch)
2019-03-17 02:10:04.450758: step 3340, examples 167000, loss = 0.837910831 (33.854 examples/sec; 1.477 sec/batch)
2019-03-17 02:10:19.331327: step 3350, examples 167500, loss = 0.808128595 (34.273 examples/sec; 1.459 sec/batch)
2019-03-17 02:10:34.363299: step 3360, examples 168000, loss = 0.674990118 (33.046 examples/sec; 1.513 sec/batch)
2019-03-17 02:10:49.353158: step 3370, examples 168500, loss = 0.764808238 (33.969 examples/sec; 1.472 sec/batch)
2019-03-17 02:11:04.259797: step 3380, examples 169000, loss = 0.719039619 (32.829 examples/sec; 1.523 sec/batch)
2019-03-17 02:11:19.236622: step 3390, examples 169500, loss = 0.668897212 (33.112 examples/sec; 1.510 sec/batch)
2019-03-17 02:11:34.205427: step 3400, examples 170000, loss = 0.853322208 (33.739 examples/sec; 1.482 sec/batch)
Top 1 validation accuracy: 0.6320754885673523 and top 2 validation accuracy: 0.830188691

2019-03-17 02:27:07.245600: step 4000, examples 200000, loss = 0.645375192 (33.993 examples/sec; 1.471 sec/batch)
Top 1 validation accuracy: 0.6297169923782349 and top 2 validation accuracy: 0.8726415038108826
Model Saved!
2019-03-17 02:27:26.955010: step 4010, examples 200500, loss = 0.648393869 (34.726 examples/sec; 1.440 sec/batch)
2019-03-17 02:27:41.905765: step 4020, examples 201000, loss = 0.688412845 (33.740 examples/sec; 1.482 sec/batch)
2019-03-17 02:27:56.791348: step 4030, examples 201500, loss = 0.673482120 (34.367 examples/sec; 1.455 sec/batch)
2019-03-17 02:28:11.593709: step 4040, examples 202000, loss = 0.692163408 (33.178 examples/sec; 1.507 sec/batch)
2019-03-17 02:28:26.434172: step 4050, examples 202500, loss = 0.586296380 (33.468 examples/sec; 1.494 sec/batch)
2019-03-17 02:28:41.208458: step 4060, examples 203000, loss = 0.774356782 (33.923 examples/sec; 1.474 sec/batch)
2019-03-17 02:28:56.148184: step 4070, examples 203500, loss = 0.702969849 (33.112 examples/s

2019-03-17 02:44:24.859617: step 4660, examples 233000, loss = 0.648194730 (32.110 examples/sec; 1.557 sec/batch)
2019-03-17 02:44:40.486093: step 4670, examples 233500, loss = 0.571914017 (31.364 examples/sec; 1.594 sec/batch)
2019-03-17 02:44:57.437706: step 4680, examples 234000, loss = 0.556474268 (30.973 examples/sec; 1.614 sec/batch)
2019-03-17 02:45:13.804226: step 4690, examples 234500, loss = 0.711023092 (31.031 examples/sec; 1.611 sec/batch)
2019-03-17 02:45:29.448826: step 4700, examples 235000, loss = 0.893343568 (32.402 examples/sec; 1.543 sec/batch)
Top 1 validation accuracy: 0.6462264060974121 and top 2 validation accuracy: 0.8254716992378235
Model Saved!
2019-03-17 02:45:50.595520: step 4710, examples 235500, loss = 0.546377659 (32.333 examples/sec; 1.546 sec/batch)
2019-03-17 02:46:05.876221: step 4720, examples 236000, loss = 0.565506697 (32.689 examples/sec; 1.530 sec/batch)
2019-03-17 02:46:21.163569: step 4730, examples 236500, loss = 0.742836177 (32.664 examples/s

In [61]:
# Accuracy for testSet(best)
stp = np.arange(25,101,25)
stp = np.insert(stp,0,10)
best_test_acc_ss_crop = 0
best_test_ss_crop = 0
for step_size in stp:
    path = 'Path_step'+str(step_size)
    model_dir = best_val_ss_crop[path]
    start = 0
    stop = 250
    step = step_size
    time_length = 700
    time_bin = 350
    #Nb = BATCH_SIZE
    fsz = 3
    use_batchnorm = True
    use_dropout = True
    use_l2loss = True
    print('Test Accuracy using cropped data for training for step size {}:'.format(step_size))
    top_1_acc_test, top_2_acc_test = test_model_ss(model_dir,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)
    if top_1_acc_test > best_test_acc_ss_crop:
        best_test_acc_ss_crop = top_1_acc_test
        best_test_ss_crop = step_size

print('The step_size {} has the best test accuracy: {}'.format(best_test_ss_crop,best_test_acc_ss_crop))

Test Accuracy using cropped data for training for step size 10:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_step10\model.ckpt-3801
[test  step 3801] loss: 1.28686; top_1_accuracy: 0.64673; top_5_accuracy: 0.844244 (7.296 sec/batch; 121.436 instances/sec)
top_1_accuracy_test =  0.64672685 top_2_accuracy_test =  0.84424376
Test Accuracy using cropped data for training for step size 25:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_step25\model.ckpt-3501
[test  step 3501] loss: 1.28993; top_1_accuracy: 0.63205; top_5_accuracy: 0.841986 (3.615 sec/batch; 245.081 instances/sec)
top_1_accuracy_test =  0.63205415 top_2_accuracy_test =  0.8419865
Test Accuracy using cropped data for training for step size 50:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_

In [62]:
def data_loader_ss_subject(time_bin,sub_id):
    X_test = np.load("X_test.npy")
    X_test = X_test[:,0:22,0:time_bin]
    y_test = np.load("y_test.npy")
    person_test = np.load("person_test.npy")

    X_train_valid = np.load("X_train_valid.npy")
    X_train_valid = X_train_valid[:,0:22,0:time_bin]
    y_train_valid = np.load("y_train_valid.npy")
    person_train_valid = np.load("person_train_valid.npy")
    
    N_train = X_train_valid.shape[0]
    idx_train_valid = np.arange(N_train,dtype='int')

    X_train, X_val, idx_train, idx_val = train_test_split(X_train_valid, idx_train_valid, test_size=0.1, random_state=21)

    y_train = y_train_valid[idx_train]
    person_train = person_train_valid[idx_train]
    y_val = y_train_valid[idx_val]
    person_val = person_train_valid[idx_val]
    
    idx11 = np.arange(0,X_test.shape[2],2,dtype=int)
    idx21 = np.arange(1,X_test.shape[2],2,dtype=int)
    X_test_ss1 = np.take(X_test,idx11,axis=2)
    X_test_ss2 = np.take(X_test,idx21,axis=2)
    X_test = np.concatenate((X_test_ss1,X_test_ss2),axis=0)
    y_test = np.concatenate((y_test,y_test),axis=0)
    person_test = np.concatenate((person_test,person_test),axis=0)
    
    idx12 = np.arange(0,X_train.shape[2],2,dtype=int)
    idx22 = np.arange(1,X_train.shape[2],2,dtype=int)
    X_train_ss1 = np.take(X_train,idx12,axis=2)
    X_train_ss2 = np.take(X_train,idx22,axis=2)
    X_train = np.concatenate((X_train_ss1,X_train_ss2),axis=0)
    y_train = np.concatenate((y_train,y_train),axis=0)
    person_train = np.concatenate((person_train,person_train),axis=0)
    
    idx13 = np.arange(0,X_val.shape[2],2,dtype=int)
    idx23 = np.arange(1,X_val.shape[2],2,dtype=int)
    X_val_ss1 = np.take(X_val,idx13,axis=2)
    X_val_ss2 = np.take(X_val,idx23,axis=2)
    X_val = np.concatenate((X_val_ss1,X_val_ss2),axis=0)
    y_val = np.concatenate((y_val,y_val),axis=0)
    person_val = np.concatenate((person_val,person_val),axis=0)

    tl = X_train.shape[2]
    X_test = np.reshape(X_test,(-1,22,tl,1))
    X_train = np.reshape(X_train,(-1,22,tl,1))
    X_val = np.reshape(X_val,(-1,22,tl,1))
    
    if sub_id<9:
        subid_test = np.where(person_test==sub_id)
        person_test = np.take(person_test,subid_test[0],axis=0)
        X_test = np.take(X_test,subid_test[0],axis=0)
        y_test = np.take(y_test,subid_test[0],axis=0)
        subid_val = np.where(person_val==sub_id)
        person_val = np.take(person_val,subid_val[0],axis=0)
        X_val = np.take(X_val,subid_val[0],axis=0)
        y_val = np.take(y_val,subid_val[0],axis=0)
        subid_train = np.where(person_train==sub_id)
        person_train = np.take(person_train,subid_train[0],axis=0)
        X_train = np.take(X_train,subid_train[0],axis=0)
        y_train = np.take(y_train,subid_train[0],axis=0)

    label0 = 769
    new_label0 = 0
    for i in range(4):
        m1 = (y_test==label0)
        m2 = (y_train==label0)
        m3 = (y_val==label0)
        np.place(y_test,m1,new_label0)
        np.place(y_train,m2,new_label0)
        np.place(y_val,m3,new_label0)
        label0 += 1
        new_label0 += 1

    labelNames = [0,1,2,3]
    train_set = {'data': X_train, 'labels': y_train, 'person': person_train}
    test_set = {'data': X_test, 'labels': y_test, 'person': person_test}
    val_set = {'data': X_val, 'labels': y_val, 'person': person_val}
    return train_set, test_set, val_set, labelNames

def sample_batch(dataset, batch_size):
    N = dataset['data'].shape[0]
    indices = np.random.randint(N, size=batch_size)
    return {key: dataset[key][indices] for key in dataset}

def train_model_ss_subject(sub_id,target_dir,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss,num_class,max_step,log_frequency,batch_size,model_saving_freq):
    # load data
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader_ss_subject(time_length,sub_id)
    print("Dataset loading completes.")
    
    best_val_acc = 0
    best_val_path = target_dir
    


    # reset default graph
    tf.reset_default_graph()

    global_step = tf.train.get_or_create_global_step()
    
    model = Model(config, labelNames,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)

    # training setups
    saver = tf.train.Saver(max_to_keep=100)

    # Build an initialization operation to run below.
    init = tf.global_variables_initializer()
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off
    
    sess = tf.Session(config=config_proto)

    sess.run(init)

    # main loop
    for step in range(max_step):
        start_time = time.time()

        # prepare data
        train_batch = sample_batch(trainSet, batch_size)
        # feed dict
        feed_dict = {
            model.input: train_batch['data'],
            model.fine_labels: train_batch['labels'],
            model.is_training: True,
        }
        
        fetch_list = [model.optimizer_op, model.loss, model.global_step]
        _, loss_value, global_step_value = sess.run(fetch_list, feed_dict=feed_dict)
        duration = time.time() - start_time
        assert not np.isnan(loss_value), 'Model diverged with loss = NaN'

        # log
        if step % log_frequency == 0:
            num_examples_per_step = batch_size
            examples_per_sec = num_examples_per_step / duration
            sec_per_batch = duration
            format_str = (
                '%s: step %d, examples %d, loss = %.9f (%.3f examples/sec; %.3f sec/batch)'
            )
            print(
                format_str % (
                    datetime.now(), step, batch_size * step,
                    loss_value,
                    examples_per_sec, sec_per_batch
                )
            )

        # Save the model checkpoint periodically.
        if step % model_saving_freq == 0 or (step + 1) == max_step:
            fetch_list = [model.accuracy,model.top_2_accuracy]
            feed_dict = {
                model.input: valSet['data'],
                model.fine_labels: valSet['labels'],
                model.is_training: False,
            }
            val_acc_1, val_acc_2 =  sess.run(fetch_list, feed_dict=feed_dict)
            checkpoint_path = os.path.join(target_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step=int(global_step_value))
            print('Top 1 validation accuracy: {} and top 2 validation accuracy: {}'.format(val_acc_1,val_acc_2))
            print('Model Saved!')
            if val_acc_1>best_val_acc:
                best_val_acc = val_acc_1
                best_val_path = checkpoint_path+'-'+str(int(global_step_value))
    return best_val_acc,best_val_path


def test_model_ss_subject(sub_id,model_dir,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss):
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader_ss_subject(time_length,sub_id)
    print("Dataset loading completes.")
    
    # reset default graph
    tf.reset_default_graph()
    
    # create model
    
    model = Model(config, labelNames,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)
    # training setups
    saver = tf.train.Saver(max_to_keep=100)
    test_accuracy = [0, 0]
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off

    with tf.Session(config=config_proto) as session:
        session.run(tf.global_variables_initializer())
        saver.restore(session, model_dir)
        # test model
        exp_results = run_single_step(session, model, testSet, mode='test')
        test_accuracy[0] = exp_results['top_1_accuracy']
        test_accuracy[1] = exp_results['top_2_accuracy']
        print('top_1_accuracy_test = ', test_accuracy[0], 'top_2_accuracy_test = ', test_accuracy[1])
        
    top1_acc = test_accuracy[0]
    top2_acc = test_accuracy[1]
    return top1_acc,top2_acc


def run_single_step(
        session,
        model,
        batch,
        mode='test',
        log=True,
):
    # construct feed dict
    feed_dict = {
        model.input: batch['data'],
        # model.coarse_labels: batch['coarse_labels'],
        model.fine_labels: batch['labels'],
        # model.label_mapping: label_mapping,
        model.is_training: mode == 'train'
    }
    
    # select proper summary op
    if mode == 'train':
        summary_op = model.train_summary_op
    elif mode == 'val':
        summary_op = model.val_summary_op
    else:
        summary_op = model.test_summary_op
    
    # construct fetch list
    fetch_list = [model.global_step, summary_op, model.loss, model.accuracy, model.top_2_accuracy]

    # run single step
    _start_time = time.time()
    _step, _summary, _loss, _top_1, _top_2 = session.run(fetch_list, feed_dict=feed_dict)[:5]
    _end_time = time.time()
    
    # collect step statistics
    step_time = _end_time - _start_time
    batch_size = batch['data'].shape[0]
    
    # log in console
    if log:
        print(('[{:5s} step {:4d}] loss: {:.5f}; top_1_accuracy: {:.5f}; top_5_accuracy: {:5f} ' +
              '({:.3f} sec/batch; {:.3f} instances/sec)'
              ).format(mode, _step, _loss, _top_1, _top_2, 
                       step_time, batch_size / step_time))
    
    # log results to file and return statistics
    if mode == 'test':
        test_fetch_list = [model.per_class_accuracy,
                model.top_2_per_class_accuracy,
                model.confusion_matrix, 
                model.pred, model.probs]
        _top_1_c,  _top_2_c, _cm, _pred, _probs = \
                session.run(test_fetch_list, feed_dict=feed_dict)
        
        
        # Log detailed test results in pickle format
        stats = {
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2,
            "top_1_perclass_accuracy": _top_1_c,
            "top_2_perclass_accuracy": _top_2_c,
            "confusion_matrix": _cm,
            "pred": _pred,
            "probs": _probs
        }
    else:
        stats = {
            "step": _step,
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2
        }
        
    return stats


In [63]:
# Test model accuracy on each subject
subject_id = np.arange(10)
start = 0
stop = 250
step = 10
time_length = 700
time_bin = 350
#Nb = BATCH_SIZE
fsz = 3
use_batchnorm = True
use_dropout = True
use_l2loss = True
path = 'Path_step10' 
model_dir = best_val_ss_crop[path]

for sub_id in subject_id:
    if sub_id < 9:
        print('Test Accuracy for subject {}:'.format(sub_id))
    else:
        print('Test Accuracy for all subjects:')
    top_1_acc_test, top_2_acc_test = test_model_ss_subject(sub_id,model_dir,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)

Test Accuracy for subject 0:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_step10\model.ckpt-3801
[test  step 3801] loss: 1.35630; top_1_accuracy: 0.56000; top_5_accuracy: 0.870000 (0.935 sec/batch; 106.900 instances/sec)
top_1_accuracy_test =  0.56 top_2_accuracy_test =  0.87
Test Accuracy for subject 1:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_step10\model.ckpt-3801
[test  step 3801] loss: 1.65400; top_1_accuracy: 0.46000; top_5_accuracy: 0.700000 (0.963 sec/batch; 103.890 instances/sec)
top_1_accuracy_test =  0.46 top_2_accuracy_test =  0.7
Test Accuracy for subject 2:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_step10\model.ckpt-3801
[test  step 3801] loss: 1.08566; top_1_accuracy: 0.75000; top_5_accuracy: 0.850000 (0.922 sec/batch; 

In [65]:
best_val_ss_crop_subject = {}

for sub_id in range(9):
    num_cls = 4
    mstp = 1000
    lfrq = 10
    bsz = 50
    msf = 100
    tr = './trained_model_final/DCNN_reg_ss_crop_subject'+str(sub_id)
    if not os.path.exists(tr):
        os.mkdir(tr)
    start = 0
    stop = 250
    step = 10
    time_length = 700
    time_bin = 350
    fsz = 3
    use_batchnorm = True
    use_dropout = True
    use_l2loss = True
    acc = 'Accuracy_subject'+str(sub_id)
    path = 'Path_subject'+str(sub_id)

    best_val_ss_crop_subject[acc],best_val_ss_crop_subject[path] = train_model_ss_subject(sub_id,tr,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss,num_cls,mstp,lfrq,bsz,msf)

Loading datasets...
Dataset loading completes.
2019-03-17 04:50:24.442240: step 0, examples 0, loss = 1.417763352 (26.709 examples/sec; 1.872 sec/batch)
Top 1 validation accuracy: 0.1964285671710968 and top 2 validation accuracy: 0.5357142686843872
Model Saved!
2019-03-17 04:50:40.734476: step 10, examples 500, loss = 1.403492093 (33.738 examples/sec; 1.482 sec/batch)
2019-03-17 04:50:55.044546: step 20, examples 1000, loss = 1.447151184 (35.359 examples/sec; 1.414 sec/batch)
2019-03-17 04:51:09.668339: step 30, examples 1500, loss = 1.395545840 (35.337 examples/sec; 1.415 sec/batch)
2019-03-17 04:51:24.234086: step 40, examples 2000, loss = 1.442118406 (33.865 examples/sec; 1.476 sec/batch)
2019-03-17 04:51:39.238335: step 50, examples 2500, loss = 1.332412839 (33.684 examples/sec; 1.484 sec/batch)
2019-03-17 04:51:54.017830: step 60, examples 3000, loss = 1.306838751 (34.138 examples/sec; 1.465 sec/batch)
2019-03-17 04:52:08.792676: step 70, examples 3500, loss = 1.207140923 (34.329 

2019-03-17 05:08:10.512300: step 670, examples 33500, loss = 0.260676026 (30.954 examples/sec; 1.615 sec/batch)
2019-03-17 05:08:27.190650: step 680, examples 34000, loss = 0.198391914 (28.992 examples/sec; 1.725 sec/batch)
2019-03-17 05:08:44.920800: step 690, examples 34500, loss = 0.186069474 (29.043 examples/sec; 1.722 sec/batch)
2019-03-17 05:09:01.343468: step 700, examples 35000, loss = 0.195424736 (31.343 examples/sec; 1.595 sec/batch)
Top 1 validation accuracy: 0.875 and top 2 validation accuracy: 0.9821428656578064
Model Saved!
2019-03-17 05:09:19.478689: step 710, examples 35500, loss = 0.147241458 (31.186 examples/sec; 1.603 sec/batch)
2019-03-17 05:09:35.621615: step 720, examples 36000, loss = 0.142259017 (30.518 examples/sec; 1.638 sec/batch)
2019-03-17 05:09:52.041276: step 730, examples 36500, loss = 0.168194547 (31.884 examples/sec; 1.568 sec/batch)
2019-03-17 05:10:08.106997: step 740, examples 37000, loss = 0.142008677 (32.981 examples/sec; 1.516 sec/batch)
2019-03-

2019-03-17 05:26:37.621987: step 330, examples 16500, loss = 0.760827601 (31.245 examples/sec; 1.600 sec/batch)
2019-03-17 05:26:53.476146: step 340, examples 17000, loss = 0.693723857 (32.256 examples/sec; 1.550 sec/batch)
2019-03-17 05:27:10.169537: step 350, examples 17500, loss = 0.793175459 (28.610 examples/sec; 1.748 sec/batch)
2019-03-17 05:27:25.788757: step 360, examples 18000, loss = 0.592938900 (33.849 examples/sec; 1.477 sec/batch)
2019-03-17 05:27:41.035116: step 370, examples 18500, loss = 0.752324879 (34.337 examples/sec; 1.456 sec/batch)
2019-03-17 05:27:56.297102: step 380, examples 19000, loss = 0.631281078 (33.376 examples/sec; 1.498 sec/batch)
2019-03-17 05:28:11.294026: step 390, examples 19500, loss = 0.838423550 (33.732 examples/sec; 1.482 sec/batch)
2019-03-17 05:28:26.628588: step 400, examples 20000, loss = 0.686273873 (32.999 examples/sec; 1.515 sec/batch)
Top 1 validation accuracy: 0.5192307829856873 and top 2 validation accuracy: 0.7884615659713745
Model Sa

Dataset loading completes.
2019-03-17 05:43:33.313042: step 0, examples 0, loss = 1.418010473 (28.513 examples/sec; 1.754 sec/batch)
Top 1 validation accuracy: 0.1315789520740509 and top 2 validation accuracy: 0.42105263471603394
Model Saved!
2019-03-17 05:43:50.200381: step 10, examples 500, loss = 1.432673931 (33.965 examples/sec; 1.472 sec/batch)
2019-03-17 05:44:05.294719: step 20, examples 1000, loss = 1.379699707 (32.944 examples/sec; 1.518 sec/batch)
2019-03-17 05:44:20.336285: step 30, examples 1500, loss = 1.392725825 (33.435 examples/sec; 1.495 sec/batch)
2019-03-17 05:44:35.515121: step 40, examples 2000, loss = 1.366487861 (32.823 examples/sec; 1.523 sec/batch)
2019-03-17 05:44:50.561916: step 50, examples 2500, loss = 1.368790507 (33.000 examples/sec; 1.515 sec/batch)
2019-03-17 05:45:05.631634: step 60, examples 3000, loss = 1.259562373 (32.490 examples/sec; 1.539 sec/batch)
2019-03-17 05:45:20.726678: step 70, examples 3500, loss = 1.276532531 (32.404 examples/sec; 1.543

2019-03-17 06:00:09.145718: step 670, examples 33500, loss = 0.208801553 (35.040 examples/sec; 1.427 sec/batch)
2019-03-17 06:00:23.759183: step 680, examples 34000, loss = 0.142767638 (33.957 examples/sec; 1.472 sec/batch)
2019-03-17 06:00:38.389274: step 690, examples 34500, loss = 0.144345030 (34.432 examples/sec; 1.452 sec/batch)
2019-03-17 06:00:52.915850: step 700, examples 35000, loss = 0.142970651 (34.357 examples/sec; 1.455 sec/batch)
Top 1 validation accuracy: 0.7631579041481018 and top 2 validation accuracy: 0.9736841917037964
Model Saved!
2019-03-17 06:01:09.133530: step 710, examples 35500, loss = 0.119792417 (32.810 examples/sec; 1.524 sec/batch)
2019-03-17 06:01:23.873935: step 720, examples 36000, loss = 0.121464707 (34.521 examples/sec; 1.448 sec/batch)
2019-03-17 06:01:38.354915: step 730, examples 36500, loss = 0.181981415 (34.118 examples/sec; 1.466 sec/batch)
2019-03-17 06:01:53.003995: step 740, examples 37000, loss = 0.135681972 (33.571 examples/sec; 1.489 sec/ba

...

Dataset loading completes.
2019-03-17 07:22:52.873883: step 0, examples 0, loss = 1.425220609 (27.999 examples/sec; 1.786 sec/batch)
Top 1 validation accuracy: 0.4117647111415863 and top 2 validation accuracy: 0.529411792755127
Model Saved!
2019-03-17 07:23:09.724368: step 10, examples 500, loss = 1.404115200 (33.112 examples/sec; 1.510 sec/batch)
2019-03-17 07:23:24.688683: step 20, examples 1000, loss = 1.418889284 (33.577 examples/sec; 1.489 sec/batch)
2019-03-17 07:23:39.537293: step 30, examples 1500, loss = 1.303700686 (34.582 examples/sec; 1.446 sec/batch)
2019-03-17 07:23:54.576433: step 40, examples 2000, loss = 1.239197731 (33.607 examples/sec; 1.488 sec/batch)
2019-03-17 07:24:09.612953: step 50, examples 2500, loss = 1.316057682 (33.338 examples/sec; 1.500 sec/batch)
2019-03-17 07:24:24.571138: step 60, examples 3000, loss = 1.217170358 (33.125 examples/sec; 1.509 sec/batch)
2019-03-17 07:24:39.608076: step 70, examples 3500, loss = 1.043883801 (33.453 examples/sec; 1.495 s

2019-03-17 07:39:44.060038: step 670, examples 33500, loss = 0.167407289 (32.602 examples/sec; 1.534 sec/batch)
2019-03-17 07:39:59.086485: step 680, examples 34000, loss = 0.153026521 (32.275 examples/sec; 1.549 sec/batch)
2019-03-17 07:40:14.146055: step 690, examples 34500, loss = 0.156714767 (32.910 examples/sec; 1.519 sec/batch)
2019-03-17 07:40:29.259264: step 700, examples 35000, loss = 0.186934531 (33.365 examples/sec; 1.499 sec/batch)
Top 1 validation accuracy: 0.7352941036224365 and top 2 validation accuracy: 0.9411764740943909
Model Saved!
2019-03-17 07:40:45.809144: step 710, examples 35500, loss = 0.174202695 (33.956 examples/sec; 1.472 sec/batch)
2019-03-17 07:41:00.699057: step 720, examples 36000, loss = 0.162357569 (33.829 examples/sec; 1.478 sec/batch)
2019-03-17 07:41:15.677369: step 730, examples 36500, loss = 0.154429421 (34.486 examples/sec; 1.450 sec/batch)
2019-03-17 07:41:30.620043: step 740, examples 37000, loss = 0.121656634 (33.669 examples/sec; 1.485 sec/ba

2019-03-17 07:56:29.241427: step 330, examples 16500, loss = 0.259639621 (33.611 examples/sec; 1.488 sec/batch)
2019-03-17 07:56:44.239653: step 340, examples 17000, loss = 0.202803373 (33.047 examples/sec; 1.513 sec/batch)
2019-03-17 07:56:59.042305: step 350, examples 17500, loss = 0.238309920 (33.220 examples/sec; 1.505 sec/batch)
2019-03-17 07:57:13.971188: step 360, examples 18000, loss = 0.245376348 (32.765 examples/sec; 1.526 sec/batch)
2019-03-17 07:57:29.102005: step 370, examples 18500, loss = 0.189432397 (33.770 examples/sec; 1.481 sec/batch)
2019-03-17 07:57:44.198161: step 380, examples 19000, loss = 0.250453055 (32.222 examples/sec; 1.552 sec/batch)
2019-03-17 07:57:59.366965: step 390, examples 19500, loss = 0.137046039 (32.342 examples/sec; 1.546 sec/batch)
2019-03-17 07:58:14.389396: step 400, examples 20000, loss = 0.202372655 (33.088 examples/sec; 1.511 sec/batch)
Top 1 validation accuracy: 0.7142857313156128 and top 2 validation accuracy: 0.8928571343421936
Model Sa

Dataset loading completes.
2019-03-17 08:13:07.873882: step 0, examples 0, loss = 1.419539928 (28.156 examples/sec; 1.776 sec/batch)
Top 1 validation accuracy: 0.13636364042758942 and top 2 validation accuracy: 0.5454545617103577
Model Saved!
2019-03-17 08:13:24.006452: step 10, examples 500, loss = 1.388378739 (35.063 examples/sec; 1.426 sec/batch)
2019-03-17 08:13:38.591436: step 20, examples 1000, loss = 1.290118456 (34.583 examples/sec; 1.446 sec/batch)
2019-03-17 08:13:53.089177: step 30, examples 1500, loss = 1.138007998 (34.181 examples/sec; 1.463 sec/batch)
2019-03-17 08:14:07.632903: step 40, examples 2000, loss = 0.982105255 (35.184 examples/sec; 1.421 sec/batch)
2019-03-17 08:14:22.261975: step 50, examples 2500, loss = 1.073893666 (33.603 examples/sec; 1.488 sec/batch)
2019-03-17 08:14:36.923100: step 60, examples 3000, loss = 1.077718973 (33.693 examples/sec; 1.484 sec/batch)
2019-03-17 08:14:51.336340: step 70, examples 3500, loss = 0.965257883 (35.768 examples/sec; 1.398

2019-03-17 08:29:37.278588: step 670, examples 33500, loss = 0.074115947 (33.609 examples/sec; 1.488 sec/batch)
2019-03-17 08:29:51.913184: step 680, examples 34000, loss = 0.087055780 (34.085 examples/sec; 1.467 sec/batch)
2019-03-17 08:30:06.445518: step 690, examples 34500, loss = 0.081723005 (34.841 examples/sec; 1.435 sec/batch)
2019-03-17 08:30:20.966914: step 700, examples 35000, loss = 0.089398652 (34.985 examples/sec; 1.429 sec/batch)
Top 1 validation accuracy: 0.7727272510528564 and top 2 validation accuracy: 0.8863636255264282
Model Saved!
2019-03-17 08:30:37.739148: step 710, examples 35500, loss = 0.087923691 (33.046 examples/sec; 1.513 sec/batch)
2019-03-17 08:30:53.044910: step 720, examples 36000, loss = 0.082441367 (33.090 examples/sec; 1.511 sec/batch)
2019-03-17 08:31:08.654524: step 730, examples 36500, loss = 0.073451526 (32.381 examples/sec; 1.544 sec/batch)
2019-03-17 08:31:23.988951: step 740, examples 37000, loss = 0.078754276 (33.082 examples/sec; 1.511 sec/ba

In [67]:
# Test model accuracy on each subject
subject_id1 = np.arange(9)
subject_id2 = np.arange(10)
start = 0
stop = 250
step = 10
time_length = 700
time_bin = 350
#Nb = BATCH_SIZE
fsz = 3
use_batchnorm = True
use_dropout = True
use_l2loss = True

for sub_id1 in subject_id1:
    path = 'Path_subject'+str(sub_id1) 
    model_dir = best_val_ss_crop_subject[path]

    for sub_id2 in subject_id2:
        if sub_id2 < 9:
            print('Test Accuracy for subject {} trained on subject {}:'.format(sub_id2,sub_id1))
        else:
            print('Test Accuracy for all subjects trained on subject {}:'.format(sub_id1))
        top_1_acc_test, top_2_acc_test = test_model_ss_subject(sub_id2,model_dir,start,stop,step,time_length,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)

Test Accuracy for subject 0 trained on subject 0:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_subject0\model.ckpt-1000
[test  step 1000] loss: 1.49412; top_1_accuracy: 0.71000; top_5_accuracy: 0.970000 (0.967 sec/batch; 103.451 instances/sec)
top_1_accuracy_test =  0.71 top_2_accuracy_test =  0.97
Test Accuracy for subject 1 trained on subject 0:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_subject0\model.ckpt-1000
[test  step 1000] loss: 5.15868; top_1_accuracy: 0.41000; top_5_accuracy: 0.570000 (0.939 sec/batch; 106.487 instances/sec)
top_1_accuracy_test =  0.41 top_2_accuracy_test =  0.57
Test Accuracy for subject 2 trained on subject 0:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_subject0\model.ckpt-1000
[test  step 1000] loss: 1.86477

Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_subject2\model.ckpt-601
[test  step  601] loss: 4.70516; top_1_accuracy: 0.39000; top_5_accuracy: 0.560000 (1.072 sec/batch; 93.251 instances/sec)
top_1_accuracy_test =  0.39 top_2_accuracy_test =  0.56
Test Accuracy for subject 2 trained on subject 2:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_subject2\model.ckpt-601
[test  step  601] loss: 1.15686; top_1_accuracy: 0.80000; top_5_accuracy: 0.950000 (1.034 sec/batch; 96.741 instances/sec)
top_1_accuracy_test =  0.8 top_2_accuracy_test =  0.95
Test Accuracy for subject 3 trained on subject 2:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_subject2\model.ckpt-601
[test  step  601] loss: 5.06216; top_1_accuracy: 0.34000; top_5_accuracy: 0.560000 (1.034 sec/batch; 96.722

[test  step  701] loss: 5.09005; top_1_accuracy: 0.29000; top_5_accuracy: 0.490000 (1.021 sec/batch; 97.911 instances/sec)
top_1_accuracy_test =  0.29 top_2_accuracy_test =  0.49
Test Accuracy for subject 3 trained on subject 4:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_subject4\model.ckpt-701
[test  step  701] loss: 2.40200; top_1_accuracy: 0.49000; top_5_accuracy: 0.780000 (1.061 sec/batch; 94.279 instances/sec)
top_1_accuracy_test =  0.49 top_2_accuracy_test =  0.78
Test Accuracy for subject 4 trained on subject 4:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_subject4\model.ckpt-701
[test  step  701] loss: 1.77874; top_1_accuracy: 0.64894; top_5_accuracy: 0.829787 (0.970 sec/batch; 96.909 instances/sec)
top_1_accuracy_test =  0.64893615 top_2_accuracy_test =  0.82978725
Test Accuracy for subject 5 trained on subject 

top_1_accuracy_test =  0.62 top_2_accuracy_test =  0.75
Test Accuracy for subject 4 trained on subject 6:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_subject6\model.ckpt-901
[test  step  901] loss: 3.16881; top_1_accuracy: 0.60638; top_5_accuracy: 0.723404 (0.882 sec/batch; 106.542 instances/sec)
top_1_accuracy_test =  0.60638297 top_2_accuracy_test =  0.7234042
Test Accuracy for subject 5 trained on subject 6:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_subject6\model.ckpt-901
[test  step  901] loss: 5.00401; top_1_accuracy: 0.28571; top_5_accuracy: 0.540816 (1.007 sec/batch; 97.361 instances/sec)
top_1_accuracy_test =  0.2857143 top_2_accuracy_test =  0.5408163
Test Accuracy for subject 6 trained on subject 6:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final

Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_subject8\model.ckpt-901
[test  step  901] loss: 3.96138; top_1_accuracy: 0.30612; top_5_accuracy: 0.561224 (0.921 sec/batch; 106.407 instances/sec)
top_1_accuracy_test =  0.30612245 top_2_accuracy_test =  0.56122446
Test Accuracy for subject 6 trained on subject 8:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_subject8\model.ckpt-901
[test  step  901] loss: 5.37648; top_1_accuracy: 0.31000; top_5_accuracy: 0.610000 (0.926 sec/batch; 107.967 instances/sec)
top_1_accuracy_test =  0.31 top_2_accuracy_test =  0.61
Test Accuracy for subject 7 trained on subject 8:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_reg_ss_crop_subject8\model.ckpt-901
[test  step  901] loss: 2.89209; top_1_accuracy: 0.49000; top_5_accuracy: 0.790000 (0.929 se

In [70]:
for sub_id in range(9):
    trainSet,_,_,_ = data_loader_ss_subject(700,sub_id)
    num_sub = trainSet['data'].shape[0]
    print('Number of trials of subject {} : {}'.format(sub_id,num_sub))

Number of trials of subject 0 : 418
Number of trials of subject 1 : 420
Number of trials of subject 2 : 434
Number of trials of subject 3 : 428
Number of trials of subject 4 : 422
Number of trials of subject 5 : 416
Number of trials of subject 6 : 442
Number of trials of subject 7 : 408
Number of trials of subject 8 : 418
